# Train GPT-OSS 20B Coding TUI Agent

**Combined pipeline** — Tool calling SFT + agent trajectory training + proxy log extraction + IPO preference + GRPO RL.

**Target failure modes to fix:**
- Tool calling errors (invalid params, non-existent MCP servers)
- No follow-through (analysis loops, never writes code)
- Circular reasoning (repeating the same analysis)
- Context loss (forgetting task state mid-session)

**Pipeline:**
1. Tool calling SFT (rank 64 LoRA)
2. Merge → Agent SFT from proxy log trajectories (rank 128 LoRA)
3. IPO preference optimisation (decisive action > endless analysis)
4. GRPO RL (execution-grounded: compiles / tests pass / no loops)
5. Eval → Export

**Base model:** [openai/gpt-oss-20b](https://huggingface.co/openai/gpt-oss-20b) (20.9B MoE, 3.6B active)

**Data sources:**
- [glaiveai/glaive-function-calling-v2](https://huggingface.co/datasets/glaiveai/glaive-function-calling-v2) (113K)
- [Salesforce/xlam-function-calling-60k](https://huggingface.co/datasets/Salesforce/xlam-function-calling-60k) (60K)
- [NousResearch/hermes-function-calling-v1](https://huggingface.co/datasets/NousResearch/hermes-function-calling-v1)
- [xingyaoww/code-act](https://huggingface.co/datasets/xingyaoww/code-act)
- [bigcode/commitpack](https://huggingface.co/datasets/bigcode/commitpack) (50K subsample)
- [bigcode/editpackft](https://huggingface.co/datasets/bigcode/editpackft) (50K subsample)
- [Anthropic/hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf)
- [m-a-p/CodeFeedback-Filtered-Instruction](https://huggingface.co/datasets/m-a-p/CodeFeedback-Filtered-Instruction)
- MacLean AI proxy logs (real Codex CLI agent sessions — most valuable source)

## Step 0: Environment Setup

### 0.1 Mount Google Drive & Clone Repository

In [ ]:
import os

IN_COLAB = "COLAB_GPU" in os.environ or os.path.exists("/content")

DRIVE_BASE = ""
DRIVE_MODE = "local"

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    DRIVE_BASE = "/content/drive/MyDrive/gpt-oss-20b-coding-tui"
    DRIVE_MODE = "mounted"
    os.makedirs(DRIVE_BASE, exist_ok=True)

    if not os.path.exists("llm-training-pipeline"):
        !git clone https://github.com/rmarnold/llm-training-pipeline.git
    os.chdir("llm-training-pipeline")
    !git pull --ff-only
    print(f"Working directory: {os.getcwd()}")
else:
    print("Running locally (not in Colab).")
    print(f"Working directory: {os.getcwd()}")

### 0.2 Install Dependencies

In [ ]:
import subprocess, sys, os

IN_COLAB = "COLAB_GPU" in os.environ or os.path.exists("/content")

if IN_COLAB:
    # Core + GPT-OSS deps
    !pip install -q -e ".[gpt_oss]"

    # Unsloth (Colab optimised)
    !pip install -q unsloth

    # vLLM for fast inference in GRPO
    !pip install -q vllm

    # ipywidgets for config UI
    !pip install -q ipywidgets

    # Datasets for HuggingFace downloads
    !pip install -q datasets huggingface_hub

    # Code execution evaluation deps
    !pip install -q pyflakes astunparse

    print("\nDependencies installed.")
else:
    print("Assuming local dependencies are already installed.")
    print("Run: pip install -e '.[gpt_oss]'")

# Reduce CUDA memory fragmentation (helps with expert LoRA's large param count)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

### 0.3 Configure Pipeline

Toggle the form view (click the "..." menu on this cell) to see the interactive configuration panel.
Adjust settings, then **run this cell** to apply them.

In [ ]:
#@title ### Pipeline Configuration { display-mode: "form" }

#@markdown ---
#@markdown #### Core Settings

training_scope = "quick_test"  #@param ["full", "quick_test", "tool_calling_only", "skip_to_rl"] {type: "string"}
gpu_tier = "h100_80gb"  #@param ["a100_40gb", "a100_80gb", "h100_80gb"] {type: "string"}
batch_profile = "balanced"  #@param ["aggressive", "balanced", "conservative"] {type: "string"}
max_steps_override = 0  #@param {type: "integer"}

#@markdown > **Batch Profile:**
#@markdown > - **aggressive** — max GPU utilization, fastest training, may OOM on longest sequences
#@markdown > - **balanced** — recommended default, good speed with expert LoRA headroom
#@markdown > - **conservative** — safest for rank 128 or experimental configs, slowest
#@markdown >
#@markdown > *Max Steps Override: 0 = use GPU tier defaults. Set > 0 to cap all stages.*

#@markdown ---
#@markdown #### Data Sources

include_proxy_logs = True  #@param {type: "boolean"}
proxy_log_dir = ""  #@param {type: "string"}

#@markdown > *proxy_log_dir: path to MacLean AI proxy log directory (contains per-request JSON files).*

include_tool_calling = True  #@param {type: "boolean"}
include_agent_trajectories = True  #@param {type: "boolean"}

#@markdown ---
#@markdown #### Pipeline Phases

include_preference = True  #@param {type: "boolean"}
include_grpo = True  #@param {type: "boolean"}
skip_data_generation = False  #@param {type: "boolean"}

#@markdown ---
#@markdown #### Export

enable_qat_export = False  #@param {type: "boolean"}

#@markdown ---
#@markdown #### Advanced

use_service_account = False  #@param {type: "boolean"}
drive_folder_id = ""  #@param {type: "string"}

# ======================================================================
# GPU tier × batch profile presets
#
# Expert LoRA trains 23x more params than attention-only (~740M at rank 64,
# ~1.48B at rank 128). Unsloth's chunked CE and gradient offloading help,
# but batch sizes still need to be lower than attention-only training.
#
# Effective batch sizes are kept consistent across profiles by adjusting
# gradient_accumulation_steps inversely with per_device_train_batch_size.
# ======================================================================
import os, sys, json

# Base GPU configs (non-batch settings)
GPU_BASE = {
    "a100_40gb": {
        "tool_calling_max_steps": 3000, "tool_calling_seq_len": 4096,
        "agent_sft_max_steps": 2000, "agent_sft_seq_len": 8192,
        "ipo_max_steps": 1000, "ipo_seq_len": 4096,
        "grpo_max_steps": 3000, "grpo_seq_len": 16384, "grpo_num_gen": 4,
        "eval_num_samples": 100,
        "load_mode": "4bit", "moe_backend": "triton", "fast_inference": False,
    },
    "a100_80gb": {
        "tool_calling_max_steps": -1, "tool_calling_seq_len": 8192,
        "agent_sft_max_steps": -1, "agent_sft_seq_len": 16384,
        "ipo_max_steps": -1, "ipo_seq_len": 8192,
        "grpo_max_steps": 5000, "grpo_seq_len": 32768, "grpo_num_gen": 4,
        "eval_num_samples": 200,
        "load_mode": "4bit", "moe_backend": "triton", "fast_inference": False,
    },
    "h100_80gb": {
        "tool_calling_max_steps": -1, "tool_calling_seq_len": 8192,
        "agent_sft_max_steps": -1, "agent_sft_seq_len": 16384,
        "ipo_max_steps": -1, "ipo_seq_len": 8192,
        "grpo_max_steps": 5000, "grpo_seq_len": 65536, "grpo_num_gen": 4,
        "eval_num_samples": 200,
        "load_mode": "fp8", "moe_backend": "triton", "fast_inference": True,
    },
}

# Batch profiles per GPU tier: (batch, grad_accum) tuples
# Effective batch = batch × grad_accum (kept ~constant within each tier)
BATCH_PROFILES = {
    "h100_80gb": {
        #                          tool_call    agent_sft    ipo          grpo
        #                          b   ga  eff  b   ga  eff  b   ga  eff  b   ga  eff
        "aggressive": {            # Fastest — max GPU util, may OOM on longest seqs
            "tool_calling_batch": 10, "tool_calling_grad_accum": 5,   # eff=50
            "agent_sft_batch": 4,    "agent_sft_grad_accum": 6,      # eff=24
            "ipo_batch": 4,          "ipo_grad_accum": 8,            # eff=32
            "grpo_batch": 2,         "grpo_grad_accum": 8,           # eff=16
        },
        "balanced": {              # Recommended — good speed + expert LoRA headroom
            "tool_calling_batch": 8,  "tool_calling_grad_accum": 6,  # eff=48
            "agent_sft_batch": 2,    "agent_sft_grad_accum": 12,     # eff=24
            "ipo_batch": 2,          "ipo_grad_accum": 16,           # eff=32
            "grpo_batch": 1,         "grpo_grad_accum": 16,          # eff=16
        },
        "conservative": {          # Safest — for rank 128 or long-context experiments
            "tool_calling_batch": 4,  "tool_calling_grad_accum": 12, # eff=48
            "agent_sft_batch": 1,    "agent_sft_grad_accum": 24,     # eff=24
            "ipo_batch": 1,          "ipo_grad_accum": 32,           # eff=32
            "grpo_batch": 1,         "grpo_grad_accum": 16,          # eff=16
        },
    },
    "a100_80gb": {
        "aggressive": {
            "tool_calling_batch": 4,  "tool_calling_grad_accum": 8,  # eff=32
            "agent_sft_batch": 2,    "agent_sft_grad_accum": 4,      # eff=8
            "ipo_batch": 2,          "ipo_grad_accum": 8,            # eff=16
            "grpo_batch": 1,         "grpo_grad_accum": 8,           # eff=8
        },
        "balanced": {
            "tool_calling_batch": 2,  "tool_calling_grad_accum": 16, # eff=32
            "agent_sft_batch": 1,    "agent_sft_grad_accum": 8,      # eff=8
            "ipo_batch": 1,          "ipo_grad_accum": 16,           # eff=16
            "grpo_batch": 1,         "grpo_grad_accum": 8,           # eff=8
        },
        "conservative": {
            "tool_calling_batch": 1,  "tool_calling_grad_accum": 32, # eff=32
            "agent_sft_batch": 1,    "agent_sft_grad_accum": 8,      # eff=8
            "ipo_batch": 1,          "ipo_grad_accum": 16,           # eff=16
            "grpo_batch": 1,         "grpo_grad_accum": 8,           # eff=8
        },
    },
    "a100_40gb": {
        "aggressive": {
            "tool_calling_batch": 2,  "tool_calling_grad_accum": 16, # eff=32
            "agent_sft_batch": 1,    "agent_sft_grad_accum": 8,      # eff=8
            "ipo_batch": 1,          "ipo_grad_accum": 16,           # eff=16
            "grpo_batch": 1,         "grpo_grad_accum": 8,           # eff=8
        },
        "balanced": {
            "tool_calling_batch": 2,  "tool_calling_grad_accum": 16, # eff=32
            "agent_sft_batch": 1,    "agent_sft_grad_accum": 8,      # eff=8
            "ipo_batch": 1,          "ipo_grad_accum": 16,           # eff=16
            "grpo_batch": 1,         "grpo_grad_accum": 8,           # eff=8
        },
        "conservative": {
            "tool_calling_batch": 1,  "tool_calling_grad_accum": 32, # eff=32
            "agent_sft_batch": 1,    "agent_sft_grad_accum": 8,      # eff=8
            "ipo_batch": 1,          "ipo_grad_accum": 16,           # eff=16
            "grpo_batch": 1,         "grpo_grad_accum": 8,           # eff=8
        },
    },
}

# Merge base config + batch profile
tier = {**GPU_BASE[gpu_tier], **BATCH_PROFILES[gpu_tier][batch_profile]}

# ======================================================================
# Build CONFIG dict from form values
# ======================================================================
CONFIG = {
    "training_scope": training_scope,
    "gpu_tier": gpu_tier,
    "batch_profile": batch_profile,
    **tier,
    # Data sources
    "include_proxy_logs": include_proxy_logs,
    "proxy_log_dir": proxy_log_dir,
    "include_tool_calling": include_tool_calling,
    "include_agent_trajectories": include_agent_trajectories,
    # Pipeline phases
    "include_preference": include_preference,
    "include_grpo": include_grpo,
    "enable_qat_export": enable_qat_export,
    "skip_data_generation": skip_data_generation,
    # Advanced
    "use_service_account": use_service_account,
    "drive_folder_id": drive_folder_id,
}

# Apply max_steps override
if max_steps_override > 0:
    for key in list(CONFIG.keys()):
        if key.endswith("_max_steps"):
            CONFIG[key] = max_steps_override

# Quick test caps
if CONFIG["training_scope"] == "quick_test":
    for key in list(CONFIG.keys()):
        if key.endswith("_max_steps"):
            CONFIG[key] = min(CONFIG[key], 50)
    CONFIG["eval_num_samples"] = 10

# Scope-based overrides
if CONFIG["training_scope"] == "tool_calling_only":
    CONFIG["include_preference"] = False
    CONFIG["include_grpo"] = False
    CONFIG["include_agent_trajectories"] = False
elif CONFIG["training_scope"] == "skip_to_rl":
    CONFIG["include_tool_calling"] = False
    CONFIG["include_agent_trajectories"] = False

# ======================================================================
# Set up DriveHelper
# ======================================================================
sys.path.insert(0, "scripts")
from pipeline_lib.drive_utils import DriveHelper

if "DRIVE_BASE" not in dir():
    DRIVE_BASE = ""
if "DRIVE_MODE" not in dir():
    DRIVE_MODE = "local"

if CONFIG["use_service_account"] and CONFIG["drive_folder_id"]:
    sa_path = "service_account.json"
    try:
        from google.colab import userdata
        sa_json = userdata.get("SERVICE_ACCOUNT_JSON")
        with open(sa_path, "w") as f:
            f.write(sa_json)
    except Exception:
        pass

    if os.path.exists(sa_path) and os.path.getsize(sa_path) > 10:
        try:
            drive_helper = DriveHelper(
                mode="service_account",
                credentials_path=sa_path,
                folder_id=CONFIG["drive_folder_id"],
            )
            DRIVE_MODE = "service_account"
        except Exception as e:
            print(f"Service account failed: {e}")
            drive_helper = DriveHelper(mode="local")
            DRIVE_MODE = "local"
    else:
        drive_helper = DriveHelper(mode="local")
        DRIVE_MODE = "local"
elif DRIVE_BASE:
    drive_helper = DriveHelper(mode="mounted", drive_base=DRIVE_BASE)
    DRIVE_MODE = "mounted"
else:
    drive_helper = DriveHelper(mode="local")
    DRIVE_MODE = "local"

# Save for persistence across restarts
os.makedirs("data", exist_ok=True)
with open("data/config_coding_tui.json", "w") as f:
    json.dump(CONFIG, f, indent=2)

# ======================================================================
# Print summary
# ======================================================================
# Compute effective batch sizes for display
tc_eff = CONFIG["tool_calling_batch"] * CONFIG["tool_calling_grad_accum"]
as_eff = CONFIG["agent_sft_batch"] * CONFIG["agent_sft_grad_accum"]
ip_eff = CONFIG["ipo_batch"] * CONFIG["ipo_grad_accum"]
gr_eff = CONFIG["grpo_batch"] * CONFIG["grpo_grad_accum"]

profile_label = {
    "aggressive": "AGGRESSIVE (fastest, may OOM)",
    "balanced": "BALANCED (recommended)",
    "conservative": "CONSERVATIVE (safest, slowest)",
}[batch_profile]

print("=" * 62)
print("  PIPELINE CONFIGURATION (Coding TUI Agent)")
print("=" * 62)
print(f"  Scope:           {CONFIG['training_scope'].upper()}")
print(f"  GPU tier:        {CONFIG['gpu_tier']}")
print(f"  Batch profile:   {profile_label}")
print(f"  MoE backend:     {CONFIG['moe_backend']}")
print(f"  Load mode:       {CONFIG['load_mode']}")
print(f"  Drive mode:      {DRIVE_MODE}")
print()
print(f"  Proxy logs:      {CONFIG['include_proxy_logs']}")
if CONFIG["include_proxy_logs"] and CONFIG["proxy_log_dir"]:
    print(f"    Log dir:       {CONFIG['proxy_log_dir']}")
print(f"  Tool calling:    {CONFIG['include_tool_calling']}")
print(f"  Agent traj:      {CONFIG['include_agent_trajectories']}")
print(f"  IPO preference:  {CONFIG['include_preference']}")
print(f"  GRPO:            {CONFIG['include_grpo']}")
print(f"  QAT export:      {CONFIG['enable_qat_export']}")
if CONFIG["skip_data_generation"]:
    print(f"  DATA GEN:        *** SKIPPED (skip_data_generation=True) ***")
if max_steps_override > 0:
    print(f"  Max steps:       {max_steps_override} (override)")
print()
print(f"  Tool Calling SFT: batch={CONFIG['tool_calling_batch']} x grad={CONFIG['tool_calling_grad_accum']} = {tc_eff}, seq={CONFIG['tool_calling_seq_len']}, steps={CONFIG['tool_calling_max_steps']}")
print(f"  Agent SFT:        batch={CONFIG['agent_sft_batch']} x grad={CONFIG['agent_sft_grad_accum']} = {as_eff}, seq={CONFIG['agent_sft_seq_len']}, steps={CONFIG['agent_sft_max_steps']}")
if CONFIG["include_preference"]:
    print(f"  IPO:              batch={CONFIG['ipo_batch']} x grad={CONFIG['ipo_grad_accum']} = {ip_eff}, seq={CONFIG['ipo_seq_len']}, steps={CONFIG['ipo_max_steps']}")
if CONFIG["include_grpo"]:
    print(f"  GRPO:             batch={CONFIG['grpo_batch']} x grad={CONFIG['grpo_grad_accum']} = {gr_eff}, seq={CONFIG['grpo_seq_len']}, steps={CONFIG['grpo_max_steps']}")
print("=" * 62)

### 0.4 Pipeline Dashboard

In [ ]:
import ipywidgets as widgets
from IPython.display import display

class PipelineTracker:
    """Track pipeline progress with visual indicators."""

    PHASES = [
        ("tool_calling_data", "Tool Calling Data"),
        ("agent_traj_data", "Agent Trajectory Data"),
        ("proxy_log_extract", "Proxy Log Extraction"),
        ("tool_calling_sft", "Tool Calling SFT"),
        ("merge", "Merge Adapter"),
        ("agent_sft", "Agent SFT"),
        ("ipo", "IPO Preference"),
        ("grpo", "GRPO RL"),
        ("eval", "Evaluation"),
        ("export", "Export"),
    ]

    def __init__(self):
        self._bars = {}
        self._labels = {}
        rows = []
        for key, name in self.PHASES:
            label = widgets.HTML(
                value=f"<span style='color:#888'>&#x25CB; {name}</span>",
                layout=widgets.Layout(width="240px"),
            )
            bar = widgets.FloatProgress(
                value=0, min=0, max=1.0,
                bar_style="info",
                layout=widgets.Layout(width="300px", height="18px"),
            )
            self._bars[key] = bar
            self._labels[key] = label
            rows.append(widgets.HBox([label, bar]))
        self._container = widgets.VBox(rows)
        display(widgets.HTML("<b>Pipeline Progress</b>"))
        display(self._container)

    def start(self, phase):
        self._labels[phase].value = (
            f"<span style='color:#2196F3'>&#x25B6; {dict(self.PHASES)[phase]}</span>"
        )
        self._bars[phase].value = 0.1
        self._bars[phase].bar_style = "info"

    def complete(self, phase):
        self._labels[phase].value = (
            f"<span style='color:#4CAF50'>&#x2714; {dict(self.PHASES)[phase]}</span>"
        )
        self._bars[phase].value = 1.0
        self._bars[phase].bar_style = "success"

    def skip(self, phase):
        self._labels[phase].value = (
            f"<span style='color:#9E9E9E'>&#x2014; {dict(self.PHASES)[phase]} (skipped)</span>"
        )
        self._bars[phase].value = 1.0
        self._bars[phase].bar_style = ""

    def fail(self, phase):
        self._labels[phase].value = (
            f"<span style='color:#F44336'>&#x2718; {dict(self.PHASES)[phase]}</span>"
        )
        self._bars[phase].bar_style = "danger"

tracker = PipelineTracker()

# Quality gate results — populated after each training phase
gate_report = {
    "tool_calling": {"passed": None, "loss": None, "notes": []},
    "agent_sft": {"passed": None, "loss": None, "notes": []},
    "ipo": {"passed": None, "loss": None, "notes": []},
    "grpo": {"passed": None, "loss": None, "notes": []},
}


### 0.5 Set Up Persistent Storage

In [ ]:
import os

DRIVE_SUBDIRS = [
    "data/coding_tui/tool_calling",
    "data/coding_tui/agent_traj",
    "data/coding_tui/proxy_logs",
    "data/coding_tui/preference",
    "data/coding_tui/grpo",
    "data/coding_tui/eval",
    "checkpoints/tool_calling_sft",
    "checkpoints/gpt-oss-20b-coding-tui-merged",
    "checkpoints/agent_sft",
    "checkpoints/agent_sft_ipo",
    "checkpoints/agent_sft_grpo",
    "evals",
]

if DRIVE_MODE == "mounted":
    for subdir in DRIVE_SUBDIRS:
        drive_path = os.path.join(DRIVE_BASE, subdir)
        os.makedirs(drive_path, exist_ok=True)
        local_path = subdir
        if not os.path.exists(local_path):
            os.makedirs(os.path.dirname(local_path) or ".", exist_ok=True)
            os.symlink(drive_path, local_path)
            print(f"  Linked: {local_path} -> {drive_path}")
        else:
            print(f"  Exists: {local_path}")
    print(f"\nDrive base: {DRIVE_BASE}")
elif DRIVE_MODE == "service_account":
    for subdir in DRIVE_SUBDIRS:
        os.makedirs(subdir, exist_ok=True)
        drive_helper.ensure_dir(subdir)
    print("Drive directories created (service account mode).")
else:
    for subdir in DRIVE_SUBDIRS:
        os.makedirs(subdir, exist_ok=True)
    print("Local directories created (no Drive backup).")

### 0.6 Check GPU & Configure MoE Backend

In [ ]:
import torch, os

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / (1024**3)

    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_mem:.1f} GB")

    # Auto-detect GPU tier override
    detected_tier = None
    if "H100" in gpu_name or "H200" in gpu_name:
        detected_tier = "h100_80gb"
    elif "A100" in gpu_name:
        detected_tier = "a100_80gb" if gpu_mem > 45 else "a100_40gb"

    if detected_tier and detected_tier != CONFIG["gpu_tier"]:
        print(f"\n  Auto-override: {CONFIG['gpu_tier']} -> {detected_tier}")
        CONFIG["gpu_tier"] = detected_tier
        tier = GPU_CONFIGS[detected_tier]
        for k, v in tier.items():
            CONFIG[k] = v
        print(f"  Updated CONFIG with {detected_tier} presets.")

    # Set MoE backend
    os.environ["UNSLOTH_MOE_BACKEND"] = CONFIG.get("moe_backend", "triton")
    print(f"\n  MoE backend: {os.environ['UNSLOTH_MOE_BACKEND']}")
    print(f"  Load mode: {CONFIG['load_mode']}")
    print(f"  Fast inference: {CONFIG.get('fast_inference', False)}")

    # FP8 detection
    if CONFIG["load_mode"] == "fp8":
        try:
            import transformer_engine
            print("  FP8: transformer-engine available")
        except ImportError:
            print("  FP8: transformer-engine not found, falling back to 4bit")
            CONFIG["load_mode"] = "4bit"
else:
    print("No GPU detected! Training will fail.")
    print("Enable GPU: Runtime -> Change runtime type -> GPU")

print(f"\nFinal config: scope={CONFIG['training_scope']}, tier={CONFIG['gpu_tier']}")

## Step 1: Data Preparation

### 1.1 Download Tool Calling Datasets

Downloads and formats three tool/function calling datasets into Harmony format:
- **Glaive v2** (113K): high-quality synthetic function calling conversations
- **xLAM-60K** (60K): diverse function calling from Salesforce
- **Hermes v1**: NousResearch curated function calling data

In [ ]:
import os, json, sys

sys.path.insert(0, "scripts")
from dataset_formatters.function_calling import (
    format_glaive_function_calling,
    format_hermes_function_calling,
)
from dataset_formatters.harmony import encode_harmony_messages

TOOL_CALLING_DATASETS = [
    ("glaiveai/glaive-function-calling-v2", "glaive", 113000),
    ("Salesforce/xlam-function-calling-60k", "xlam", 60000),
    ("NousResearch/hermes-function-calling-v1", "hermes", None),
]

# ──────────────────────────────────────────────────────────────────────────────
# xLAM inline formatter
# xLAM format: {"query": str, "tools": str (JSON array), "answers": str (JSON array)}
# Output: Harmony tool call format
# ──────────────────────────────────────────────────────────────────────────────
def format_xlam_function_calling(example):
    """Format Salesforce xLAM function calling data into Harmony format.

    xLAM schema:
        query   - natural language user request
        tools   - JSON-encoded list of tool schemas (name/description/parameters)
        answers - JSON-encoded list of call dicts [{name: ..., arguments: {...}}]

    Returns Harmony-encoded text with developer context, user query, and
    one tool_call per answer entry in the assistant turn.
    """
    query = example.get("query", "").strip()
    tools_raw = example.get("tools", "[]")
    answers_raw = example.get("answers", "[]")

    if not query:
        return {"text": ""}

    try:
        tools = json.loads(tools_raw) if isinstance(tools_raw, str) else tools_raw
    except (json.JSONDecodeError, TypeError):
        tools = []

    try:
        answers = json.loads(answers_raw) if isinstance(answers_raw, str) else answers_raw
    except (json.JSONDecodeError, TypeError):
        answers = []

    if not answers:
        return {"text": ""}

    # Build tool schema description for developer context
    tool_descriptions = []
    for t in tools:
        name = t.get("name", "unknown")
        desc = t.get("description", "")
        params = t.get("parameters", {})
        tool_descriptions.append(
            f"  {name}: {desc}\n    Parameters: {json.dumps(params, separators=(',', ':'))}"
        )
    tool_ctx = "\n".join(tool_descriptions) if tool_descriptions else "No tools defined."

    dev_instructions = (
        "You are a helpful assistant with access to tools. "
        "Call the appropriate tool with valid parameters based on the user's request.\n\n"
        f"Available tools:\n{tool_ctx}"
    )

    # Build tool_calls list from answers
    tool_calls = []
    for i, ans in enumerate(answers):
        tc_name = ans.get("name", "")
        tc_args = ans.get("arguments", ans.get("parameters", {}))
        if not tc_name:
            continue
        tool_calls.append({
            "id": f"call_{i}",
            "type": "function",
            "function": {
                "name": tc_name,
                "arguments": json.dumps(tc_args) if isinstance(tc_args, dict) else str(tc_args),
            },
        })

    if not tool_calls:
        return {"text": ""}

    messages = [
        {"role": "user", "content": query},
        {"role": "assistant", "tool_calls": tool_calls},
    ]

    return {"text": encode_harmony_messages(messages, developer_instructions=dev_instructions)}


if CONFIG["skip_data_generation"] or not CONFIG["include_tool_calling"]:
    print("Skipping tool calling data download.")
    tracker.skip("tool_calling_data")
else:
    tracker.start("tool_calling_data")
    from datasets import load_dataset, Dataset, concatenate_datasets

    all_tool_calling = []
    stats = {}

    for ds_name, ds_key, max_samples in TOOL_CALLING_DATASETS:
        print(f"\nDownloading {ds_name}...")
        try:
            if ds_key == "glaive":
                raw = load_dataset(ds_name, split="train")
                if CONFIG["training_scope"] == "quick_test":
                    raw = raw.select(range(min(500, len(raw))))
                elif max_samples:
                    raw = raw.select(range(min(max_samples, len(raw))))
                formatted = raw.map(format_glaive_function_calling, remove_columns=raw.column_names)

            elif ds_key == "xlam":
                raw = load_dataset(ds_name, split="train")
                if CONFIG["training_scope"] == "quick_test":
                    raw = raw.select(range(min(200, len(raw))))
                elif max_samples:
                    raw = raw.select(range(min(max_samples, len(raw))))
                formatted = raw.map(format_xlam_function_calling, remove_columns=raw.column_names)

            elif ds_key == "hermes":
                raw = load_dataset(ds_name, split="train")
                if CONFIG["training_scope"] == "quick_test":
                    raw = raw.select(range(min(300, len(raw))))
                formatted = raw.map(format_hermes_function_calling, remove_columns=raw.column_names)

            else:
                continue

            # Filter empty examples
            formatted = formatted.filter(lambda x: bool(x.get("text", "").strip()))
            stats[ds_key] = len(formatted)
            all_tool_calling.append(formatted)
            print(f"  Formatted: {len(formatted):,} examples")

        except Exception as e:
            print(f"  WARNING: failed to load {ds_name}: {e}")
            stats[ds_key] = 0

    if all_tool_calling:
        combined = concatenate_datasets(all_tool_calling)
        combined = combined.shuffle(seed=42)
        out_path = "data/coding_tui/tool_calling/train"
        combined.save_to_disk(out_path)
        print(f"\nTotal tool calling examples: {len(combined):,} -> {out_path}")
    else:
        print("WARNING: no tool calling data collected.")

    drive_helper.backup("data/coding_tui/tool_calling", "data/coding_tui/tool_calling")
    if DRIVE_MODE != "local":
        print("Backed up to Drive.")

    tracker.complete("tool_calling_data")

### 1.2 Download Agent Trajectory Datasets

Downloads and formats agent trajectory datasets into Harmony agent format:
- **code-act**: multi-step code execution agent trajectories
- **commitpack**: commit-based code change examples
- **editpackft**: instruction-following code edits

In [ ]:
import os, sys, json

sys.path.insert(0, "scripts")
from dataset_formatters.harmony import format_harmony_agent, encode_harmony_messages

CODING_AGENT_DEV_PROMPT = (
    "You are a coding agent. Use tools to read files, write code, run tests, and "
    "complete programming tasks. Do not just analyze — always take action and produce "
    "working code. After making changes, verify they work by running the relevant tests. "
    "If a tool call fails, diagnose and retry with corrected parameters."
)


def format_code_act_example(example):
    """Format code-act dataset into Harmony agent trajectory format."""
    conversations = example.get("conversations", [])
    if not conversations:
        return {"text": ""}

    messages = []
    for turn in conversations:
        role = turn.get("role", turn.get("from", ""))
        content = turn.get("content", turn.get("value", ""))
        if not content:
            continue
        if role in ("human", "user"):
            messages.append({"role": "user", "content": content})
        elif role in ("gpt", "assistant"):
            messages.append({"role": "assistant", "content": content})
        elif role in ("tool", "function", "observation"):
            messages.append({"role": "tool", "content": content})

    if len(messages) < 2:
        return {"text": ""}

    return {"text": encode_harmony_messages(
        messages,
        developer_instructions=CODING_AGENT_DEV_PROMPT,
        reasoning_effort="high",
    )}


def format_commitpackft_example(example):
    """Format commitpackft (commit message + code diff) into Harmony format."""
    subject = example.get("subject", "")
    message = example.get("message", subject)
    old_code = example.get("old_contents", "")
    new_code = example.get("new_contents", "")

    if not old_code or not new_code or not message:
        return {"text": ""}
    if old_code.strip() == new_code.strip():
        return {"text": ""}
    if len(old_code) > 8000 or len(new_code) > 8000:
        return {"text": ""}

    user_content = (
        f"Apply the following change to the code:\n\n"
        f"Commit message: {message}\n\n"
        f"Current code:\n```\n{old_code}\n```"
    )
    assistant_content = f"```\n{new_code}\n```"

    messages = [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": assistant_content},
    ]

    return {"text": encode_harmony_messages(
        messages,
        developer_instructions=CODING_AGENT_DEV_PROMPT,
        reasoning_effort="medium",
    )}


def format_editpackft_example(example):
    """Format nuprl/EditPackFT (instruction-following code edits) into Harmony format."""
    instruction = example.get("instruction", "")
    old_code = example.get("old_contents", example.get("input", ""))
    new_code = example.get("new_contents", example.get("output", ""))

    if not instruction or not old_code or not new_code:
        return {"text": ""}
    if old_code.strip() == new_code.strip():
        return {"text": ""}
    if len(old_code) > 8000 or len(new_code) > 8000:
        return {"text": ""}

    user_content = (
        f"{instruction}\n\n"
        f"Code:\n```\n{old_code}\n```"
    )
    assistant_content = f"```\n{new_code}\n```"

    messages = [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": assistant_content},
    ]

    return {"text": encode_harmony_messages(
        messages,
        developer_instructions=CODING_AGENT_DEV_PROMPT,
        reasoning_effort="medium",
    )}


if CONFIG["skip_data_generation"] or not CONFIG["include_agent_trajectories"]:
    print("Skipping agent trajectory data download.")
    tracker.skip("agent_traj_data")
else:
    tracker.start("agent_traj_data")
    from datasets import load_dataset, Dataset, concatenate_datasets

    all_agent_traj = []

    # 1. xingyaoww/code-act — splits are "codeact" and "general", NOT "train"
    print("\nDownloading xingyaoww/code-act...")
    try:
        codeact_split = load_dataset("xingyaoww/code-act", split="codeact")
        general_split = load_dataset("xingyaoww/code-act", split="general")
        raw = concatenate_datasets([codeact_split, general_split])
        print(f"  Loaded: {len(codeact_split):,} codeact + {len(general_split):,} general = {len(raw):,} total")
        if CONFIG["training_scope"] == "quick_test":
            raw = raw.select(range(min(200, len(raw))))
        formatted = raw.map(format_code_act_example, remove_columns=raw.column_names)
        formatted = formatted.filter(lambda x: bool(x.get("text", "").strip()))
        all_agent_traj.append(formatted)
        print(f"  Formatted: {len(formatted):,} examples")
    except Exception as e:
        print(f"  WARNING: failed to load xingyaoww/code-act: {e}")

    # 2. bigcode/commitpackft — Parquet-based replacement for commitpack (script loading deprecated)
    #    Load multiple languages for coding diversity
    print("\nDownloading bigcode/commitpackft...")
    try:
        commitpack_parts = []
        languages = ["python", "javascript", "go", "rust", "java", "typescript"]
        per_lang = 10000 if CONFIG["training_scope"] != "quick_test" else 50
        for lang in languages:
            try:
                lang_ds = load_dataset("bigcode/commitpackft", lang, split="train")
                lang_ds = lang_ds.select(range(min(per_lang, len(lang_ds))))
                commitpack_parts.append(lang_ds)
                print(f"  {lang}: {len(lang_ds):,} examples")
            except Exception as lang_e:
                print(f"  {lang}: skipped ({lang_e})")
        if commitpack_parts:
            raw = concatenate_datasets(commitpack_parts)
            formatted = raw.map(format_commitpackft_example, remove_columns=raw.column_names)
            formatted = formatted.filter(lambda x: bool(x.get("text", "").strip()))
            all_agent_traj.append(formatted)
            print(f"  Formatted: {len(formatted):,} examples total")
    except Exception as e:
        print(f"  WARNING: failed to load bigcode/commitpackft: {e}")

    # 3. nuprl/EditPackFT — correct path (not bigcode/editpackft which doesn't exist)
    print("\nDownloading nuprl/EditPackFT...")
    try:
        raw = load_dataset("nuprl/EditPackFT", split="train")
        print(f"  Loaded: {len(raw):,} examples")
        if CONFIG["training_scope"] == "quick_test":
            raw = raw.select(range(min(300, len(raw))))
        formatted = raw.map(format_editpackft_example, remove_columns=raw.column_names)
        formatted = formatted.filter(lambda x: bool(x.get("text", "").strip()))
        all_agent_traj.append(formatted)
        print(f"  Formatted: {len(formatted):,} examples")
    except Exception as e:
        print(f"  WARNING: failed to load nuprl/EditPackFT: {e}")

    if all_agent_traj:
        combined = concatenate_datasets(all_agent_traj)
        combined = combined.shuffle(seed=42)
        out_path = "data/coding_tui/agent_traj/train"
        combined.save_to_disk(out_path)
        print(f"\nTotal agent trajectory examples: {len(combined):,} -> {out_path}")
    else:
        print("WARNING: no agent trajectory data collected.")

    drive_helper.backup("data/coding_tui/agent_traj", "data/coding_tui/agent_traj")
    if DRIVE_MODE != "local":
        print("Backed up to Drive.")

    tracker.complete("agent_traj_data")


### 1.3 Download Preference Datasets

Downloads and formats preference datasets for IPO training.
Good responses (task completed, decisive action) are paired against bad ones (circular analysis, incomplete).

In [ ]:
import os, sys, json, re

sys.path.insert(0, "scripts")
from dataset_formatters.harmony import format_harmony_preference, encode_harmony_messages

CODING_PREF_DEV = "You are a helpful coding assistant. Provide complete, working code solutions."


def format_hh_rlhf_example(example):
    """Format Anthropic HH-RLHF into Harmony preference pairs.

    hh-rlhf format: {"chosen": str, "rejected": str}
    Both are conversation strings with \\n\\nHuman: / \\n\\nAssistant: turns.

    Returns only "text" key to avoid column name conflicts with remove_columns.
    The chosen/rejected/prompt fields are stored as JSON in a "metadata" column.
    """
    chosen_raw = example.get("chosen") or ""
    rejected_raw = example.get("rejected") or ""

    if not chosen_raw or not rejected_raw:
        return {"text": "", "pref_chosen": "", "pref_rejected": "", "pref_prompt": ""}

    def parse_conversation(raw):
        messages = []
        parts = re.split(r'\n\n(Human|Assistant):\s*', raw)
        current_role = None
        for part in parts:
            stripped = part.strip()
            if stripped == "Human":
                current_role = "user"
            elif stripped == "Assistant":
                current_role = "assistant"
            elif stripped and current_role:
                messages.append({"role": current_role, "content": stripped})
                current_role = None
        return messages

    chosen_msgs = parse_conversation(chosen_raw)
    rejected_msgs = parse_conversation(rejected_raw)

    if not chosen_msgs or not rejected_msgs:
        return {"text": "", "pref_chosen": "", "pref_rejected": "", "pref_prompt": ""}

    prompt_msgs = chosen_msgs[:-1] if len(chosen_msgs) > 1 else []
    chosen_content = chosen_msgs[-1].get("content", "") if chosen_msgs else ""
    rejected_content = rejected_msgs[-1].get("content", "") if rejected_msgs else ""

    if not chosen_content or not rejected_content or chosen_content == rejected_content:
        return {"text": "", "pref_chosen": "", "pref_rejected": "", "pref_prompt": ""}

    chosen_full = encode_harmony_messages(
        prompt_msgs + [{"role": "assistant", "content": chosen_content}],
        developer_instructions=CODING_PREF_DEV,
    )
    rejected_full = encode_harmony_messages(
        prompt_msgs + [{"role": "assistant", "content": rejected_content}],
        developer_instructions=CODING_PREF_DEV,
    )

    prompt_text = prompt_msgs[-1].get("content", "") if prompt_msgs else ""

    # Use prefixed column names to avoid conflicts with original "chosen"/"rejected" columns
    return {
        "text": chosen_full,
        "pref_prompt": prompt_text,
        "pref_chosen": chosen_full,
        "pref_rejected": rejected_full,
    }


def format_code_feedback_example(example):
    """Format CodeFeedback-Filtered-Instruction into Harmony preference pairs.

    Schema: {"query": str, "answer": str, "resource": str, "lang": str}
    """
    query = (example.get("query") or example.get("instruction") or "").strip()
    answer = (example.get("answer") or example.get("output") or "").strip()

    if not query or not answer or len(answer) < 100:
        return {"text": "", "pref_chosen": "", "pref_rejected": "", "pref_prompt": ""}

    # Synthetic rejected: truncate + non-committal ending (trains model to prefer completion)
    cutoff = max(50, int(len(answer) * 0.3))
    rejected = answer[:cutoff] + "\n\n(I would need to analyze this further before proceeding.)"

    chosen_msgs = [
        {"role": "user", "content": query},
        {"role": "assistant", "content": answer},
    ]
    rejected_msgs = [
        {"role": "user", "content": query},
        {"role": "assistant", "content": rejected},
    ]

    chosen_full = encode_harmony_messages(chosen_msgs, developer_instructions=CODING_PREF_DEV)
    rejected_full = encode_harmony_messages(rejected_msgs, developer_instructions=CODING_PREF_DEV)

    return {
        "text": chosen_full,
        "pref_prompt": query,
        "pref_chosen": chosen_full,
        "pref_rejected": rejected_full,
    }


if CONFIG["skip_data_generation"] or not CONFIG["include_preference"]:
    print("Skipping preference data download.")
else:
    from datasets import load_dataset, Dataset, concatenate_datasets

    all_pref = []

    # 1. Anthropic/hh-rlhf
    print("\nDownloading Anthropic/hh-rlhf...")
    try:
        raw = load_dataset("Anthropic/hh-rlhf", split="train")
        print(f"  Loaded: {len(raw):,} examples")
        if CONFIG["training_scope"] == "quick_test":
            raw = raw.select(range(min(300, len(raw))))
        # Use batched=False and don't remove original columns to avoid name conflicts
        formatted = raw.map(format_hh_rlhf_example)
        # Now drop original columns manually
        cols_to_keep = ["text", "pref_prompt", "pref_chosen", "pref_rejected"]
        cols_to_drop = [c for c in formatted.column_names if c not in cols_to_keep]
        if cols_to_drop:
            formatted = formatted.remove_columns(cols_to_drop)
        formatted = formatted.filter(lambda x: bool(x.get("text", "").strip()))
        all_pref.append(formatted)
        print(f"  Formatted: {len(formatted):,} preference pairs")
    except Exception as e:
        print(f"  WARNING: failed to load Anthropic/hh-rlhf: {e}")
        import traceback; traceback.print_exc()

    # 2. m-a-p/CodeFeedback-Filtered-Instruction
    print("\nDownloading m-a-p/CodeFeedback-Filtered-Instruction...")
    try:
        raw = load_dataset("m-a-p/CodeFeedback-Filtered-Instruction", split="train")
        print(f"  Loaded: {len(raw):,} examples")
        if CONFIG["training_scope"] == "quick_test":
            raw = raw.select(range(min(200, len(raw))))
        formatted = raw.map(format_code_feedback_example)
        cols_to_keep = ["text", "pref_prompt", "pref_chosen", "pref_rejected"]
        cols_to_drop = [c for c in formatted.column_names if c not in cols_to_keep]
        if cols_to_drop:
            formatted = formatted.remove_columns(cols_to_drop)
        formatted = formatted.filter(lambda x: bool(x.get("text", "").strip()))
        all_pref.append(formatted)
        print(f"  Formatted: {len(formatted):,} preference pairs")
    except Exception as e:
        print(f"  WARNING: failed to load m-a-p/CodeFeedback-Filtered-Instruction: {e}")
        import traceback; traceback.print_exc()

    if all_pref:
        combined = concatenate_datasets(all_pref)
        combined = combined.shuffle(seed=42)
        out_path = "data/coding_tui/preference/train"
        combined.save_to_disk(out_path)
        print(f"\nTotal preference examples: {len(combined):,} -> {out_path}")
    else:
        print("WARNING: no preference data collected.")

    drive_helper.backup("data/coding_tui/preference", "data/coding_tui/preference")
    if DRIVE_MODE != "local":
        print("Backed up to Drive.")


### 1.4 Extract Training Data from Proxy Logs (Optional)

Scans the MacLean AI proxy log directory for real Codex CLI agent sessions.
Each session is a JSON file written by `claude-proxy-v2` containing the full
request/response. These are the highest-value training examples because they
are real agent tasks, not synthetic data.

Also scans for `*_cot.txt` files (chain-of-thought logs from the streaming filter)
and associates them with the corresponding requests for thinking data.

**Set `proxy_log_dir` in Step 0.3 to enable.**

In [ ]:
import os, sys, json, glob, re
from pathlib import Path

sys.path.insert(0, "scripts")
from dataset_formatters.harmony import encode_harmony_messages

CODING_AGENT_DEV_PROMPT = (
    "You are a coding agent. Use tools to read files, write code, run tests, and "
    "complete programming tasks. Do not just analyze — always take action and produce "
    "working code. After making changes, verify they work by running the relevant tests. "
    "If a tool call fails, diagnose and retry with corrected parameters."
)


def load_proxy_log(log_path):
    """Load and validate a single proxy log JSON file.

    Returns the parsed dict if valid, None otherwise.
    Expected fields: request_num, path, translated, original_request,
                     response, input_tokens, output_tokens, latency_ms
    """
    try:
        with open(log_path) as f:
            data = json.load(f)
    except (json.JSONDecodeError, OSError):
        return None

    # Must have a response with output
    if data.get("output_tokens", 0) <= 0:
        return None

    resp = data.get("response", {})
    if not resp:
        return None

    return data


def extract_messages_from_request(req_data, cot_text=None):
    """Extract Harmony-ready messages from a proxy log entry.

    Handles both:
    - /v1/messages (Anthropic format, possibly translated)
    - /v1/chat/completions (OpenAI format)

    Args:
        req_data: parsed proxy log dict
        cot_text: optional chain-of-thought text from *_cot.txt sidecar

    Returns list of message dicts suitable for encode_harmony_messages, or None.
    """
    path = req_data.get("path", "")
    original_req = req_data.get("original_request", {})
    response = req_data.get("response", {})

    messages = []

    if "/messages" in path:
        # Anthropic Messages API format
        sys_content = original_req.get("system", "")
        if isinstance(sys_content, list):
            # system can be a list of content blocks
            sys_text = " ".join(
                block.get("text", "") for block in sys_content
                if isinstance(block, dict) and block.get("type") == "text"
            )
        else:
            sys_text = str(sys_content) if sys_content else ""

        raw_msgs = original_req.get("messages", [])
        for m in raw_msgs:
            role = m.get("role", "")
            content = m.get("content", "")
            if isinstance(content, list):
                # content can be a list of blocks (text, tool_use, tool_result)
                text_parts = []
                tool_calls_out = []
                for block in content:
                    btype = block.get("type", "")
                    if btype == "text":
                        text_parts.append(block.get("text", ""))
                    elif btype == "tool_use":
                        tool_calls_out.append({
                            "id": block.get("id", ""),
                            "type": "function",
                            "function": {
                                "name": block.get("name", ""),
                                "arguments": json.dumps(block.get("input", {})),
                            },
                        })
                    elif btype == "tool_result":
                        # tool results come back as user messages in Anthropic format
                        result_content = block.get("content", "")
                        if isinstance(result_content, list):
                            result_content = " ".join(
                                rb.get("text", "") for rb in result_content
                                if isinstance(rb, dict)
                            )
                        messages.append({
                            "role": "tool",
                            "tool_call_id": block.get("tool_use_id", ""),
                            "content": result_content,
                        })

                msg_entry = {"role": role}
                if text_parts:
                    msg_entry["content"] = "\n".join(text_parts)
                if tool_calls_out:
                    msg_entry["tool_calls"] = tool_calls_out
                if role == "assistant" and cot_text:
                    msg_entry["thinking"] = cot_text
                    cot_text = None  # use only for the first assistant turn

                if role not in ("user",) or text_parts:
                    messages.append(msg_entry)
            else:
                entry = {"role": role, "content": str(content) if content else ""}
                if role == "assistant" and cot_text:
                    entry["thinking"] = cot_text
                    cot_text = None
                messages.append(entry)

        # Extract assistant response
        resp_content = response.get("content", [])
        if isinstance(resp_content, list):
            resp_text_parts = []
            resp_tool_calls = []
            for block in resp_content:
                btype = block.get("type", "")
                if btype == "text":
                    resp_text_parts.append(block.get("text", ""))
                elif btype == "tool_use":
                    resp_tool_calls.append({
                        "id": block.get("id", ""),
                        "type": "function",
                        "function": {
                            "name": block.get("name", ""),
                            "arguments": json.dumps(block.get("input", {})),
                        },
                    })
            resp_entry = {"role": "assistant"}
            if resp_text_parts:
                resp_entry["content"] = "\n".join(resp_text_parts)
            if resp_tool_calls:
                resp_entry["tool_calls"] = resp_tool_calls
            messages.append(resp_entry)
        elif isinstance(resp_content, str) and resp_content:
            messages.append({"role": "assistant", "content": resp_content})

        dev_instructions = sys_text if sys_text else CODING_AGENT_DEV_PROMPT

    elif "/chat/completions" in path:
        # OpenAI Chat Completions format
        raw_msgs = original_req.get("messages", [])
        dev_instructions = CODING_AGENT_DEV_PROMPT

        for m in raw_msgs:
            role = m.get("role", "")
            content = m.get("content", "")
            tool_calls = m.get("tool_calls", [])
            tool_call_id = m.get("tool_call_id")

            if role == "system":
                dev_instructions = content
                continue

            entry = {"role": role}
            if content:
                entry["content"] = content
            if tool_calls:
                entry["tool_calls"] = tool_calls
            if tool_call_id:
                entry["tool_call_id"] = tool_call_id
            if role == "assistant" and cot_text:
                entry["thinking"] = cot_text
                cot_text = None
            messages.append(entry)

        # Extract response from choices
        choices = response.get("choices", [])
        if choices:
            resp_msg = choices[0].get("message", {})
            resp_entry = {"role": "assistant"}
            if resp_msg.get("content"):
                resp_entry["content"] = resp_msg["content"]
            if resp_msg.get("tool_calls"):
                resp_entry["tool_calls"] = resp_msg["tool_calls"]
            messages.append(resp_entry)
    else:
        return None, None

    if len(messages) < 2:
        return None, None

    return messages, dev_instructions


def extract_proxy_log_trajectories(log_dir, max_samples=None, quick_test=False):
    """Scan proxy log directory and extract training examples.

    Args:
        log_dir: path to MacLean AI proxy log directory
        max_samples: cap on number of examples to extract
        quick_test: if True, stop after 20 examples

    Returns list of Harmony-encoded text strings.
    """
    log_dir = Path(log_dir)
    if not log_dir.exists():
        print(f"  WARNING: proxy log dir not found: {log_dir}")
        return []

    # Find all JSON log files (exclude *_cot.txt sidecars)
    log_files = sorted(log_dir.glob("*.json"))
    if not log_files:
        # Try subdirectories (MacLean AI organises logs by date)
        log_files = sorted(log_dir.glob("**/*.json"))

    print(f"  Found {len(log_files)} log files in {log_dir}")

    examples = []
    skipped = 0

    for log_path in log_files:
        if quick_test and len(examples) >= 20:
            break
        if max_samples and len(examples) >= max_samples:
            break

        req_data = load_proxy_log(log_path)
        if req_data is None:
            skipped += 1
            continue

        # Check for CoT sidecar: same stem but _cot.txt suffix
        cot_path = log_path.with_name(log_path.stem + "_cot.txt")
        cot_text = None
        if cot_path.exists():
            try:
                cot_text = cot_path.read_text(encoding="utf-8").strip() or None
            except OSError:
                pass

        messages, dev_instructions = extract_messages_from_request(req_data, cot_text)
        if messages is None:
            skipped += 1
            continue

        try:
            text = encode_harmony_messages(
                messages,
                developer_instructions=dev_instructions,
                reasoning_effort="high",
            )
        except Exception as e:
            skipped += 1
            continue

        if text and len(text.strip()) > 200:
            examples.append({"text": text})

    print(f"  Extracted: {len(examples):,} valid examples ({skipped} skipped)")
    return examples


# ── Run extraction ─────────────────────────────────────────────────────────────
log_dir = CONFIG.get("proxy_log_dir", "").strip()

if not CONFIG["include_proxy_logs"] or not log_dir:
    print("Proxy log extraction disabled (include_proxy_logs=False or proxy_log_dir not set).")
    print("To enable: set proxy_log_dir to the MacLean AI log directory in Step 0.3.")
    tracker.skip("proxy_log_extract")
elif CONFIG["skip_data_generation"]:
    print("Skipping proxy log extraction (skip_data_generation=True).")
    tracker.skip("proxy_log_extract")
else:
    tracker.start("proxy_log_extract")

    quick = CONFIG["training_scope"] == "quick_test"
    proxy_examples = extract_proxy_log_trajectories(
        log_dir,
        max_samples=5000 if not quick else None,
        quick_test=quick,
    )

    if proxy_examples:
        from datasets import Dataset, load_from_disk, concatenate_datasets

        proxy_ds = Dataset.from_list(proxy_examples)
        proxy_ds = proxy_ds.shuffle(seed=42)

        # Save standalone proxy dataset
        proxy_out = "data/coding_tui/proxy_logs/train"
        proxy_ds.save_to_disk(proxy_out)
        print(f"\nProxy log dataset: {len(proxy_ds):,} examples -> {proxy_out}")

        # Merge into agent_sft data (proxy logs are highest value)
        agent_path = "data/coding_tui/agent_traj/train"
        if os.path.exists(agent_path):
            base_ds = load_from_disk(agent_path)
            merged = concatenate_datasets([base_ds, proxy_ds])
            merged = merged.shuffle(seed=42)
            merged.save_to_disk(agent_path)
            print(f"Merged into agent_traj/train: {len(merged):,} total")
        else:
            proxy_ds.save_to_disk(agent_path)
            print(f"Saved as agent_traj/train: {len(proxy_ds):,} examples")

        drive_helper.backup("data/coding_tui/proxy_logs", "data/coding_tui/proxy_logs")
        if DRIVE_MODE != "local":
            print("Backed up to Drive.")

        tracker.complete("proxy_log_extract")
    else:
        print("No valid proxy log examples extracted.")
        tracker.fail("proxy_log_extract")

### 1.5 Verify Data

In [ ]:
import os

data_checks = [
    ("Tool Calling train", "data/coding_tui/tool_calling/train"),
    ("Agent Trajectory train", "data/coding_tui/agent_traj/train"),
    ("Proxy Log train", "data/coding_tui/proxy_logs/train"),
    ("Preference train", "data/coding_tui/preference/train"),
]

print("Data Verification Summary")
print("=" * 60)
print(f"  {'Dataset':<30} {'Examples':>12}")
print("-" * 60)

total = 0
for name, path in data_checks:
    if os.path.exists(path):
        try:
            from datasets import load_from_disk
            ds = load_from_disk(path)
            count = len(ds)
            total += count
            print(f"  {name:<30} {count:>12,}")
        except Exception as e:
            print(f"  {name:<30} {'ERROR':>12}  ({e})")
    else:
        print(f"  {name:<30} {'not found':>12}")

print("-" * 60)
print(f"  {'TOTAL':<30} {total:>12,}")
print("=" * 60)

### 1.6 Verify MoE Expert LoRA Targeting

GPT-OSS 20B uses MoE (Mixture of Experts) with fused FFN layers.
Unsloth Bug #3405: default target modules (`gate_proj`, `up_proj`, `down_proj`) silently miss expert layers.
This cell loads the model briefly to verify that LoRA adapters will be applied to expert FFN layers,
not just attention layers. If experts are NOT targeted, training only adapts ~31.8M params instead of ~200M+.

In [ ]:
import sys, torch, os
sys.path.insert(0, "scripts")

print("=" * 60)
print("  MoE Expert LoRA Diagnostic")
print("=" * 60)

from pipeline_lib.unsloth_utils import (
    load_unsloth_model, detect_moe_experts, get_moe_expert_regex,
    apply_lora_config, verify_expert_lora, print_trainable_params,
)

# Quick load to inspect architecture (will be reloaded during training)
print("\nLoading model for architecture inspection...")
model, tokenizer = load_unsloth_model(
    model_name="openai/gpt-oss-20b",
    max_seq_length=2048,  # Short — just inspecting, not training
    load_in_4bit=True,
)

# Step 1: Detect MoE experts
print("\n--- Step 1: MoE Architecture Detection ---")
moe_info = detect_moe_experts(model)
print(f"  Is MoE: {moe_info['is_moe']}")
print(f"  Expert layer types: {moe_info['expert_module_names']}")
print(f"  Number of experts: {moe_info['num_experts']}")
print(f"  Expert param count: {moe_info['expert_param_count']:,}")
if moe_info.get('sample_param_names'):
    print(f"  Sample param names:")
    for n in moe_info['sample_param_names']:
        print(f"    {n}")

# Step 2: Get expert regex
print("\n--- Step 2: Expert Regex Pattern ---")
regex = get_moe_expert_regex(model)
print(f"  Regex: {regex}")

# Step 3: Apply LoRA (tries Unsloth+regex first, PEFT direct fallback)
print("\n--- Step 3: Apply LoRA (Unsloth regex → PEFT direct fallback) ---")
test_config = {
    "r": 8,  # Small rank — just testing, not training
    "lora_alpha": 16,
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_up_proj", "down_proj"],
    "lora_dropout": 0,
    "bias": "none",
    "use_gradient_checkpointing": "unsloth",
    "use_rslora": False,
}
model = apply_lora_config(model, test_config, auto_detect_moe=True)

print("\n--- Step 4: Verification ---")
result = verify_expert_lora(model)
print_trainable_params(model)

if result["has_expert_lora"]:
    print("\n  PASS: Expert FFN layers have LoRA adapters.")
    print(f"  {result['expert_lora_count']} expert LoRA parameters detected.")
    print(f"  {result['expert_lora_params']:,} expert trainable params.")
    print("  Training will update expert routing knowledge (not just attention).")
else:
    print("\n  FAIL: Expert FFN layers do NOT have LoRA adapters!")
    print("  Dumping all trainable parameter names for debugging:")
    for name in result["all_lora_names"][:20]:
        print(f"    {name}")
    if len(result["all_lora_names"]) > 20:
        print(f"    ... ({len(result['all_lora_names']) - 20} more)")

# Clean up to free VRAM before real training
del model, tokenizer
torch.cuda.empty_cache()
import gc; gc.collect()
print("\nDiagnostic complete. Model unloaded to free VRAM.")

## Step 2: Tool Calling SFT (Phase 1)

Train a LoRA adapter (rank 64) focused on correct tool/function calling.

**Goals:**
- Learn parameter accuracy for tool calls
- Learn when NOT to call tools
- Learn valid tool schemas and JSON formatting
- Reduce hallucinated MCP server calls

Low rank (64) to avoid catastrophic forgetting of general coding ability.

### 2.1 Train Tool Calling Adapter

In [ ]:
if CONFIG["training_scope"] in ("skip_to_rl",):
    print(f"Skipping — scope is {CONFIG['training_scope']}")
    tracker.skip("tool_calling_sft")
elif not CONFIG["include_tool_calling"]:
    print("Skipping — include_tool_calling=False")
    tracker.skip("tool_calling_sft")
else:
    tracker.start("tool_calling_sft")

    batch = CONFIG["tool_calling_batch"]
    grad_accum = CONFIG["tool_calling_grad_accum"]
    max_steps = CONFIG["tool_calling_max_steps"]
    seq_len = CONFIG["tool_calling_seq_len"]
    epochs = 1  # Single pass for tool calling (large dataset)

    cmd = "python scripts/13_train_lang_adapter.py"
    cmd += " --train_data_path data/coding_tui/tool_calling/train"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"
    cmd += f" --num_train_epochs {epochs}"
    cmd += f" --output_dir checkpoints/tool_calling_sft"

    print("Training tool calling SFT adapter...")
    print(f"  Data:     data/coding_tui/tool_calling/train")
    print(f"  Batch:    {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Epochs:   {epochs}")
    print(f"  Max steps: {max_steps} (-1 = epoch-controlled)")
    print(f"  Seq len:  {seq_len}")
    print(f"  LoRA rank: 64 (low rank to preserve general ability)")
    print(f"  MoE backend: {CONFIG['moe_backend']}")
    print("=" * 60)

    !{cmd}

    drive_helper.backup("checkpoints/tool_calling_sft", "checkpoints/tool_calling_sft")
    if DRIVE_MODE != "local":
        print("\nCheckpoint backed up to Drive.")

    tracker.complete("tool_calling_sft")

### 2.2 Merge Tool Calling Adapter into Base

In [ ]:
if CONFIG["training_scope"] in ("skip_to_rl",):
    print(f"Skipping — scope is {CONFIG['training_scope']}")
    tracker.skip("merge")
elif not CONFIG["include_tool_calling"]:
    print("Skipping — include_tool_calling=False")
    tracker.skip("merge")
else:
    tracker.start("merge")

    print("Merging tool calling adapter into base model...")
    print("=" * 60)

    !python scripts/19_merge_adapter.py \
        --adapter_path checkpoints/tool_calling_sft/final \
        --output_dir checkpoints/gpt-oss-20b-coding-tui-merged

    drive_helper.backup(
        "checkpoints/gpt-oss-20b-coding-tui-merged",
        "checkpoints/gpt-oss-20b-coding-tui-merged",
    )
    if DRIVE_MODE != "local":
        print("\nMerged model backed up to Drive.")

    tracker.complete("merge")

### 2.3 Verify Merge

In [ ]:
import os, json, glob

if CONFIG["training_scope"] not in ("skip_to_rl",) and CONFIG["include_tool_calling"]:
    print("=" * 60)
    print("  QUALITY GATE: Tool Calling SFT")
    print("=" * 60)
    gate = gate_report["tool_calling"]
    gate["passed"] = True

    # 1. Check training loss
    state_files = glob.glob("checkpoints/tool_calling_sft/**/trainer_state.json", recursive=True)
    if state_files:
        with open(sorted(state_files)[-1]) as f:
            state = json.load(f)
        log_history = state.get("log_history", [])
        train_losses = [e["loss"] for e in log_history if "loss" in e]
        if train_losses:
            gate["loss"] = train_losses[-1]
            avg_last = sum(train_losses[-10:]) / len(train_losses[-10:])
            gate["avg_loss"] = avg_last
            gate["first_loss"] = train_losses[0]
            gate["total_steps"] = len(train_losses)
            improvement = (train_losses[0] - avg_last) / train_losses[0] * 100
            gate["improvement"] = improvement
            print(f"  Loss (first):     {train_losses[0]:.4f}")
            print(f"  Loss (final):     {train_losses[-1]:.4f}")
            print(f"  Loss (avg last):  {avg_last:.4f}")
            print(f"  Improvement:      {improvement:.1f}%")
            print(f"  Total steps:      {len(train_losses)}")
            if avg_last > 2.0:
                gate["passed"] = False
                gate["notes"].append(f"Loss too high ({avg_last:.4f} > 2.0)")
                print(f"  ✗ Loss too high")
            elif avg_last > 1.5:
                gate["notes"].append(f"Loss elevated ({avg_last:.4f} > 1.5)")
                print(f"  ⚠ Loss elevated")
            else:
                print(f"  ✓ Loss acceptable")
    else:
        gate["passed"] = False
        gate["notes"].append("No trainer_state.json found")
        print("  ✗ No trainer_state.json found")

    # 2. Check merged model
    merged_path = "checkpoints/gpt-oss-20b-coding-tui-merged"
    if os.path.exists(merged_path):
        files = os.listdir(merged_path)
        total_size = sum(
            os.path.getsize(os.path.join(merged_path, f))
            for f in files if os.path.isfile(os.path.join(merged_path, f))
        )
        size_gb = total_size / (1024**3)
        print(f"  Merged model:     {size_gb:.1f} GB ({len(files)} files)")
        if size_gb < 5.0:
            gate["passed"] = False
            gate["notes"].append(f"Merged model too small ({size_gb:.1f} GB)")
        else:
            print(f"  ✓ Merged model OK")
    else:
        gate["notes"].append("Merged model not found (will use base)")
        print(f"  ⚠ Merged model not found — agent SFT will use base model")

    # 3. Quick inference test
    print("\n  Running inference smoke test...")
    try:
        from unsloth import FastLanguageModel
        test_model_path = merged_path if os.path.exists(merged_path) else "checkpoints/tool_calling_sft/final"
        model, tokenizer = FastLanguageModel.from_pretrained(
            test_model_path, max_seq_length=4096, load_in_4bit=True,
        )
        FastLanguageModel.for_inference(model)

        test_prompt = (
            "<|developer|>\nYou are a coding agent with tools: run_command, read_file, apply_patch.\n"
            "<|user|>\nRead the file src/main.rs\n"
            "<|assistant|>\n"
        )
        inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.1, do_sample=True)
        response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=False)

        gate["inference_response"] = response[:500]
        has_tool = any(tok in response for tok in ["read_file", "run_command", "<|tool_call|>", "function"])
        print(f"  Response: {response[:200]}")
        if has_tool:
            print(f"  ✓ Produces tool calls")
        else:
            gate["notes"].append("No tool call in inference test")
            print(f"  ⚠ No tool call detected")

        del model, tokenizer
        import gc; gc.collect()
        import torch; torch.cuda.empty_cache()
    except Exception as e:
        gate["notes"].append(f"Inference test failed: {e}")
        print(f"  ⚠ Inference test skipped: {e}")

    status = "✅ PASSED" if gate["passed"] else "❌ FAILED"
    print(f"\n  {status}")
    if gate["notes"]:
        for note in gate["notes"]:
            print(f"    → {note}")
    print("=" * 60)
    print("  Continuing to next phase regardless...")
else:
    print("Tool calling SFT skipped.")


## Step 3: Agent SFT (Phase 2)

Train a higher-rank LoRA (rank 128) on agent trajectories using the merged
tool-calling model as the base.

**Data includes:**
- Multi-turn code agent sessions (code-act, commitpack, editpackft)
- Real proxy log trajectories from live Codex CLI sessions (most valuable)

**Goals:**
- Learn complete read → plan → edit → verify cycles
- Learn to actually write code after planning
- Learn context tracking across long sessions

### 3.1 Train Agent SFT Adapter

In [ ]:
if CONFIG["training_scope"] in ("tool_calling_only", "skip_to_rl"):
    print(f"Skipping — scope is {CONFIG['training_scope']}")
    tracker.skip("agent_sft")
elif not CONFIG["include_agent_trajectories"]:
    print("Skipping — include_agent_trajectories=False")
    tracker.skip("agent_sft")
else:
    tracker.start("agent_sft")

    # Use merged model as base if available, otherwise base model
    import os
    if os.path.exists("checkpoints/gpt-oss-20b-coding-tui-merged"):
        base_model = "checkpoints/gpt-oss-20b-coding-tui-merged"
        print("Using merged tool-calling model as base.")
    else:
        base_model = "openai/gpt-oss-20b"
        print("Merged model not found, using original base model.")

    batch = CONFIG["agent_sft_batch"]
    grad_accum = CONFIG["agent_sft_grad_accum"]
    max_steps = CONFIG["agent_sft_max_steps"]
    seq_len = CONFIG["agent_sft_seq_len"]
    epochs = 3  # Agent behavior needs more passes

    cmd = "python scripts/14_train_core_agent.py"
    cmd += " --train_data_path data/coding_tui/agent_traj/train"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"
    cmd += f" --num_train_epochs {epochs}"
    cmd += f" --output_dir checkpoints/agent_sft"

    print("Training agent SFT adapter (rank 128)...")
    print(f"  Base:     {base_model}")
    print(f"  Data:     data/coding_tui/agent_traj/train")
    print(f"  Batch:    {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Epochs:   {epochs}")
    print(f"  Max steps: {max_steps} (-1 = epoch-controlled)")
    print(f"  Seq len:  {seq_len}")
    print(f"  LoRA rank: 128")
    print(f"  MoE backend: {CONFIG['moe_backend']}")
    print(f"  Auto packing: enabled")
    print("=" * 60)

    !{cmd}

    drive_helper.backup("checkpoints/agent_sft", "checkpoints/agent_sft")
    if DRIVE_MODE != "local":
        print("\nCheckpoint backed up to Drive.")

    tracker.complete("agent_sft")

In [ ]:
import os, json, glob

if CONFIG["training_scope"] not in ("tool_calling_only", "skip_to_rl") and CONFIG["include_agent_trajectories"]:
    print("=" * 60)
    print("  QUALITY GATE: Agent SFT")
    print("=" * 60)
    gate = gate_report["agent_sft"]
    gate["passed"] = True

    # 1. Check training loss
    state_files = glob.glob("checkpoints/agent_sft/**/trainer_state.json", recursive=True)
    if state_files:
        with open(sorted(state_files)[-1]) as f:
            state = json.load(f)
        log_history = state.get("log_history", [])
        train_losses = [e["loss"] for e in log_history if "loss" in e]
        if train_losses:
            gate["loss"] = train_losses[-1]
            avg_last = sum(train_losses[-10:]) / len(train_losses[-10:])
            gate["avg_loss"] = avg_last
            gate["first_loss"] = train_losses[0]
            gate["total_steps"] = len(train_losses)
            improvement = (train_losses[0] - avg_last) / train_losses[0] * 100
            gate["improvement"] = improvement
            print(f"  Loss (first):     {train_losses[0]:.4f}")
            print(f"  Loss (final):     {train_losses[-1]:.4f}")
            print(f"  Loss (avg last):  {avg_last:.4f}")
            print(f"  Improvement:      {improvement:.1f}%")
            print(f"  Total steps:      {len(train_losses)}")
            if avg_last > 1.8:
                gate["passed"] = False
                gate["notes"].append(f"Loss too high ({avg_last:.4f} > 1.8)")
            elif improvement < 10:
                gate["notes"].append(f"Low improvement ({improvement:.1f}% < 10%)")
                print(f"  ⚠ Low convergence")
            else:
                print(f"  ✓ Good convergence")
    else:
        gate["passed"] = False
        gate["notes"].append("No trainer_state.json found")

    # 2. Check checkpoint and LoRA config
    ckpt_path = "checkpoints/agent_sft/final"
    if os.path.exists(ckpt_path):
        adapter_config = os.path.join(ckpt_path, "adapter_config.json")
        if os.path.exists(adapter_config):
            with open(adapter_config) as f:
                cfg = json.load(f)
            rank = cfg.get("r", 0)
            gate["lora_rank"] = rank
            print(f"  LoRA rank: {rank}, alpha: {cfg.get('lora_alpha', '?')}")
            if rank != 128:
                gate["notes"].append(f"Unexpected rank {rank} (expected 128)")
        print(f"  ✓ Checkpoint exists ({len(os.listdir(ckpt_path))} files)")
    else:
        gate["passed"] = False
        gate["notes"].append("Checkpoint not found")

    # 3. Behavior test
    if os.path.exists(ckpt_path):
        print("\n  Running agent behavior test...")
        try:
            from unsloth import FastLanguageModel
            model, tokenizer = FastLanguageModel.from_pretrained(
                ckpt_path, max_seq_length=8192, load_in_4bit=True,
            )
            FastLanguageModel.for_inference(model)

            test_prompt = (
                "<|developer|>\nYou are a coding agent. Use tools to complete tasks. Always take action.\n"
                "<|user|>\nFix the failing test in src/parser.rs\n"
                "<|assistant|>\nLet me read the test file first.\n"
                "<|tool_call|>\n{\"id\": \"1\", \"name\": \"read_file\", \"arguments\": {\"path\": \"src/parser.rs\"}}\n"
                "<|tool_result|>\n<|tool_call_id|>1\nfn parse(input: &str) -> Result<Ast, Error> {\n    unimplemented!()\n}\n\n#[test]\nfn test_parse_basic() {\n    assert!(parse(\"hello\").is_ok());\n}\n"
                "<|assistant|>\n"
            )
            inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
            outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.1, do_sample=True)
            response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=False)

            gate["inference_response"] = response[:500]
            print(f"  Response: {response[:300]}")

            takes_action = any(tok in response for tok in ["apply_patch", "write", "impl", "fn parse", "<|tool_call|>"])
            stalls = any(phrase in response.lower() for phrase in [
                "i would need to", "let me analyze", "i need more context",
                "further investigation", "i cannot"
            ])
            gate["takes_action"] = takes_action
            gate["stalls"] = stalls

            if takes_action:
                print(f"  ✓ Takes action")
            else:
                gate["notes"].append("No decisive action in behavior test")
                print(f"  ⚠ No decisive action detected")
            if stalls:
                gate["notes"].append("Stalling/hedging language detected")
                print(f"  ⚠ Stalling language detected")
            else:
                print(f"  ✓ No stalling")

            del model, tokenizer
            import gc; gc.collect()
            import torch; torch.cuda.empty_cache()
        except Exception as e:
            gate["notes"].append(f"Behavior test failed: {e}")
            print(f"  ⚠ Test skipped: {e}")

    status = "✅ PASSED" if gate["passed"] else "❌ FAILED"
    print(f"\n  {status}")
    if gate["notes"]:
        for note in gate["notes"]:
            print(f"    → {note}")
    print("=" * 60)
    print("  Continuing to next phase regardless...")
else:
    print("Agent SFT skipped.")


## Step 4: IPO Preference Optimisation (Phase 3)

Train with IPO on preference pairs targeting the key failure modes:

**Good (chosen):** Task completed, code written, tests pass, decisive action
**Bad (rejected):** Circular analysis, "I would need to look at this more", no code written, wrong tool params

Very low learning rate (5e-7), 1 epoch to avoid collapse.

### 4.1 Train with IPO

In [ ]:
import os

if CONFIG["training_scope"] == "tool_calling_only":
    print("Skipping — scope is tool_calling_only")
    tracker.skip("ipo")
elif not CONFIG["include_preference"]:
    print("Skipping — include_preference=False")
    tracker.skip("ipo")
else:
    tracker.start("ipo")

    batch = CONFIG["ipo_batch"]
    grad_accum = CONFIG["ipo_grad_accum"]
    max_steps = CONFIG["ipo_max_steps"]

    # Determine best checkpoint to train from
    if CONFIG["training_scope"] == "skip_to_rl":
        ipo_base = "checkpoints/agent_sft/final"
        print("skip_to_rl: starting IPO from agent_sft checkpoint")
    elif os.path.exists("checkpoints/agent_sft/final"):
        ipo_base = "checkpoints/agent_sft/final"
    else:
        ipo_base = "checkpoints/tool_calling_sft/final"
        print("agent_sft not found, falling back to tool_calling_sft")

    # Check data exists
    pref_path = "data/coding_tui/preference/train"
    if not os.path.exists(pref_path):
        print(f"WARNING: preference data not found at {pref_path}")
        print("Run Step 1.3 first.")
        tracker.fail("ipo")
    else:
        cmd = "python scripts/17_ipo_preference.py"
        cmd += f" --checkpoint {ipo_base}"
        cmd += f" --train_data_path {pref_path}"
        cmd += f" --per_device_train_batch_size {batch}"
        cmd += f" --gradient_accumulation_steps {grad_accum}"
        cmd += f" --max_steps {max_steps}"
        cmd += " --output_dir checkpoints/agent_sft_ipo"
        cmd += " --beta 0.1"

        print("Training with IPO (preference optimisation)...")
        print(f"  Base checkpoint: {ipo_base}")
        print(f"  Data:            {pref_path}")
        print(f"  Batch:           {batch} x {grad_accum} = {batch * grad_accum}")
        print(f"  Steps:           {max_steps}")
        print(f"  Loss:            IPO (beta=0.1)")
        print(f"  Load mode:       {CONFIG['load_mode']}")
        print(f"  MoE backend:     {CONFIG['moe_backend']}")
        print("=" * 60)

        !{cmd}

        drive_helper.backup("checkpoints/agent_sft_ipo", "checkpoints/agent_sft_ipo")
        if DRIVE_MODE != "local":
            print("\nCheckpoint backed up to Drive.")

        tracker.complete("ipo")

In [ ]:
import os, json, glob

if CONFIG["training_scope"] not in ("tool_calling_only",) and CONFIG["include_preference"]:
    print("=" * 60)
    print("  QUALITY GATE: IPO Preference")
    print("=" * 60)
    gate = gate_report["ipo"]
    gate["passed"] = True

    state_files = glob.glob("checkpoints/agent_sft_ipo/**/trainer_state.json", recursive=True)
    if state_files:
        with open(sorted(state_files)[-1]) as f:
            state = json.load(f)
        log_history = state.get("log_history", [])

        losses = [e.get("loss", e.get("train_loss")) for e in log_history]
        losses = [l for l in losses if l is not None]
        if losses:
            gate["loss"] = losses[-1]
            gate["first_loss"] = losses[0]
            print(f"  Loss (first):     {losses[0]:.4f}")
            print(f"  Loss (final):     {losses[-1]:.4f}")

        reward_accs = [e.get("rewards/accuracies", e.get("eval_rewards/accuracies")) for e in log_history]
        reward_accs = [r for r in reward_accs if r is not None]
        if reward_accs:
            gate["reward_accuracy"] = reward_accs[-1]
            print(f"  Reward accuracy:  {reward_accs[-1]:.3f}")
            if reward_accs[-1] < 0.55:
                gate["notes"].append(f"Reward accuracy near chance ({reward_accs[-1]:.3f})")
                print(f"  ⚠ Barely above chance")
            else:
                print(f"  ✓ Distinguishes chosen from rejected")
    else:
        gate["notes"].append("No trainer_state.json found")

    ckpt_path = "checkpoints/agent_sft_ipo/final"
    if os.path.exists(ckpt_path):
        print(f"  ✓ Checkpoint: {len(os.listdir(ckpt_path))} files")
    else:
        gate["passed"] = False
        gate["notes"].append("Checkpoint not found")
        print(f"  ✗ Checkpoint not found")

    status = "✅ PASSED" if gate["passed"] else "❌ FAILED"
    print(f"\n  {status}")
    if gate["notes"]:
        for note in gate["notes"]:
            print(f"    → {note}")
    print("=" * 60)
    print("  Continuing to next phase regardless...")
else:
    print("IPO skipped.")


## Step 5: GRPO RL (Phase 4)

Execution-grounded RL with Codex-style evaluation.

**Reward function:**
- `+1.0` for code that compiles / passes syntax check
- `+2.0` for passing test cases
- `+0.5` for clean linting (no obvious errors)
- `-1.0` for circular/no-action responses (no code written)
- `-0.5` for tool calls with malformed JSON parameters

**Goals:**
- Reinforce follow-through and complete code generation
- Penalise looping analysis without action
- Reinforce correct tool parameter formatting

**Optimisations:**
- FP8 RL with vLLM inference on H100 (1.6x throughput)
- Chunked batching for longer context
- Harmony format compliance reward

### 5.1 Train with GRPO

In [ ]:
import os

if CONFIG["training_scope"] == "tool_calling_only":
    print("Skipping — scope is tool_calling_only")
    tracker.skip("grpo")
elif not CONFIG["include_grpo"]:
    print("Skipping — include_grpo=False")
    tracker.skip("grpo")
else:
    tracker.start("grpo")

    batch = CONFIG["grpo_batch"]
    grad_accum = CONFIG["grpo_grad_accum"]
    max_steps = CONFIG["grpo_max_steps"]
    max_seq = CONFIG["grpo_seq_len"]
    num_gen = CONFIG["grpo_num_gen"]

    # Determine best checkpoint
    if os.path.exists("checkpoints/agent_sft_ipo/final"):
        grpo_base = "checkpoints/agent_sft_ipo/final"
    elif os.path.exists("checkpoints/agent_sft/final"):
        grpo_base = "checkpoints/agent_sft/final"
        print("IPO checkpoint not found, using agent_sft.")
    elif os.path.exists("checkpoints/tool_calling_sft/final"):
        grpo_base = "checkpoints/tool_calling_sft/final"
        print("agent_sft not found, using tool_calling_sft.")
    else:
        grpo_base = "openai/gpt-oss-20b"
        print("No fine-tuned checkpoint found, using base model.")

    cmd = "python scripts/18_grpo_rl.py"
    cmd += f" --checkpoint {grpo_base}"
    cmd += f" --per_device_train_batch_size {batch}"
    cmd += f" --gradient_accumulation_steps {grad_accum}"
    cmd += f" --max_steps {max_steps}"
    cmd += f" --num_generations {num_gen}"
    cmd += " --output_dir checkpoints/agent_sft_grpo"
    cmd += " --reward_mode coding_tui"
    cmd += " --developer_prompt 'You are a coding agent. Use tools to read files, write code, run tests, and complete programming tasks. Do not just analyze — always take action and produce working code.'"

    v4_features = [f"Split LoRA ({CONFIG['moe_backend']})"]
    if CONFIG["load_mode"] == "fp8":
        v4_features.append("FP8 weights")
    if CONFIG.get("fast_inference"):
        v4_features.append("vLLM inference")
    v4_features += ["Chunked batching (auto)", "Auto packing"]

    if CONFIG["gpu_tier"] == "a100_40gb":
        print("NOTE: 40GB GPU — GRPO sequence length capped at 16384")

    print("Training with GRPO (execution-grounded RL)...")
    print(f"  Base:          {grpo_base}")
    print(f"  Batch:         {batch} x {grad_accum} = {batch * grad_accum}")
    print(f"  Steps:         {max_steps}")
    print(f"  Seq length:    {max_seq}")
    print(f"  Generations:   {num_gen} per prompt")
    print()
    print("  Reward signals:")
    print("    +1.0 code compiles / passes syntax check")
    print("    +2.0 test cases pass")
    print("    +0.5 clean linting")
    print("    -1.0 circular/no-action response")
    print("    -0.5 malformed tool call JSON")
    print()
    print("  Active features:")
    for feat in v4_features:
        print(f"    ✓ {feat}")
    print("=" * 60)

    !{cmd}

    drive_helper.backup("checkpoints/agent_sft_grpo", "checkpoints/agent_sft_grpo")
    if DRIVE_MODE != "local":
        print("\nCheckpoint backed up to Drive.")

    tracker.complete("grpo")

In [ ]:
import os

if CONFIG["training_scope"] not in ("tool_calling_only",) and CONFIG["include_grpo"]:
    ckpt_path = "checkpoints/agent_sft_grpo/final"

    print("GRPO Verification:")
    print("=" * 60)

    if os.path.exists(ckpt_path):
        files = os.listdir(ckpt_path)
        print(f"  ✓ GRPO checkpoint: {ckpt_path} ({len(files)} files)")
    else:
        print(f"  ✗ GRPO checkpoint not found at {ckpt_path}")

    print("=" * 60)
else:
    print("GRPO skipped for this training scope.")

## Training Quality Report\n\nSummary of all quality gates across training phases.

In [ ]:
import json

print("=" * 70)
print("  TRAINING QUALITY REPORT")
print("=" * 70)

phases = [
    ("Tool Calling SFT", "tool_calling"),
    ("Agent SFT", "agent_sft"),
    ("IPO Preference", "ipo"),
    ("GRPO RL", "grpo"),
]

passed_count = 0
failed_count = 0
skipped_count = 0

for label, key in phases:
    gate = gate_report[key]
    if gate["passed"] is None:
        status = "⏭  SKIPPED"
        skipped_count += 1
    elif gate["passed"]:
        status = "✅ PASSED"
        passed_count += 1
    else:
        status = "❌ FAILED"
        failed_count += 1

    loss_str = f"loss={gate['loss']:.4f}" if gate.get("loss") else "no loss data"
    print(f"\n  {status}  {label} ({loss_str})")

    if gate.get("first_loss") and gate.get("loss"):
        imp = (gate["first_loss"] - gate["loss"]) / gate["first_loss"] * 100
        print(f"           Loss: {gate['first_loss']:.4f} → {gate['loss']:.4f} ({imp:+.1f}%)")
    if gate.get("total_steps"):
        print(f"           Steps: {gate['total_steps']}")
    if gate.get("reward_accuracy"):
        print(f"           Reward accuracy: {gate['reward_accuracy']:.3f}")
    if gate.get("takes_action") is not None:
        action = "✓ yes" if gate["takes_action"] else "✗ no"
        stall = "✗ yes" if gate.get("stalls") else "✓ no"
        print(f"           Takes action: {action} | Stalls: {stall}")
    if gate.get("notes"):
        for note in gate["notes"]:
            print(f"           ⚠ {note}")

print(f"\n{'=' * 70}")
print(f"  SUMMARY: {passed_count} passed, {failed_count} failed, {skipped_count} skipped")

if failed_count == 0 and passed_count > 0:
    print(f"  → All gates passed. Model ready for evaluation.")
elif failed_count > 0:
    print(f"  → {failed_count} gate(s) failed. Review notes above.")
    print(f"    The final model was still trained — evaluate it to decide")
    print(f"    whether to retrain specific phases or accept the results.")
print(f"{'=' * 70}")

# Save report to disk for reference
report_path = "evals/gate_report.json"
os.makedirs("evals", exist_ok=True)
with open(report_path, "w") as f:
    # Clean up non-serializable values
    clean_report = {}
    for k, v in gate_report.items():
        clean_report[k] = {kk: vv for kk, vv in v.items() if not callable(vv)}
    json.dump(clean_report, f, indent=2, default=str)
print(f"\nReport saved to {report_path}")


## Step 6: Evaluation

Evaluate on coding agent tasks targeting the four failure modes:
1. **Tool call format accuracy** — JSON schema compliance, valid tool names
2. **Task completion rate** — did it actually produce a code change?
3. **Circular detection rate** — does it loop the same analysis?
4. **Code correctness** — compiles, passes tests

### 6.1 Run Coding Agent Evaluation

In [ ]:
import os

if CONFIG["training_scope"] == "tool_calling_only":
    # Still run reduced eval
    pass

tracker.start("eval")

# Determine best checkpoint
CHECKPOINT_PRIORITY = [
    "checkpoints/agent_sft_grpo/final",
    "checkpoints/agent_sft_ipo/final",
    "checkpoints/agent_sft/final",
    "checkpoints/tool_calling_sft/final",
]

eval_checkpoint = None
for path in CHECKPOINT_PRIORITY:
    if os.path.exists(path):
        eval_checkpoint = path
        break

if eval_checkpoint is None:
    print("✗ No checkpoint found. Train the model first.")
    tracker.fail("eval")
else:
    num_samples = CONFIG["eval_num_samples"]

    print(f"Evaluating checkpoint: {eval_checkpoint}")
    print(f"Samples: {num_samples}")
    print("=" * 60)

    !python scripts/eval_rust_agent.py \
        --checkpoint {eval_checkpoint} \
        --num_samples {num_samples} \
        --eval_mode coding_tui \
        --output_dir evals/coding_tui_agent

    drive_helper.backup("evals/coding_tui_agent", "evals/coding_tui_agent")
    if DRIVE_MODE != "local":
        print("\nResults backed up to Drive.")

    tracker.complete("eval")

### 6.2 Check Promotion Gates

In [ ]:
!python scripts/12_check_gates.py coding_tui_agent

In [ ]:
import os, json

metrics_path = "evals/coding_tui_agent/metrics.json"

if os.path.exists(metrics_path):
    with open(metrics_path) as f:
        metrics = json.load(f)

    # Targets tuned for a coding TUI agent
    targets = {
        "tool_call_format_accuracy": (0.95, "higher"),   # Valid JSON + known tool names
        "task_completion_rate": (0.70, "higher"),         # Actually wrote/modified code
        "circular_detection_rate": (0.10, "lower"),       # Loop rate should be low
        "code_correctness_rate": (0.65, "higher"),        # Compiles / passes tests
        "follow_through_rate": (0.80, "higher"),          # Takes action after planning
    }

    print("=" * 62)
    print("EVALUATION RESULTS — Coding TUI Agent")
    print("=" * 62)
    print(f"  {'Metric':<34} {'Value':>8} {'Target':>8} {'Status':>8}")
    print("-" * 62)

    all_pass = True
    for key, (target, direction) in targets.items():
        value = metrics.get(key)
        if value is None:
            print(f"  {key:<34} {'N/A':>8} {target:>8} {'—':>8}")
            continue

        if direction == "higher":
            passed = value >= target
        else:
            passed = value <= target

        if not passed:
            all_pass = False

        status = "✓ PASS" if passed else "✗ FAIL"
        fmt_val = f"{value:.1%}" if isinstance(value, float) and value <= 1 else f"{value}"
        fmt_tgt = f"{target:.0%}" if isinstance(target, float) and target <= 1 else f"{target}"
        print(f"  {key:<34} {fmt_val:>8} {fmt_tgt:>8} {status:>8}")

    print("=" * 62)
    if all_pass:
        print("  ALL GATES PASSED ✓ — Model ready for export")
    else:
        print("  SOME GATES FAILED ✗ — Consider additional training")
    print("=" * 62)
else:
    print(f"✗ Metrics file not found at {metrics_path}")
    print("Run evaluation (6.1) first.")

## Step 7: Test Model

Load the trained model and test it interactively against the specific coding TUI
agent failure modes that this pipeline targets.

### 7.1 Load Model

In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch, os

CHECKPOINT_PRIORITY = [
    "checkpoints/agent_sft_grpo/final",
    "checkpoints/agent_sft_ipo/final",
    "checkpoints/agent_sft/final",
    "checkpoints/tool_calling_sft/final",
]

MERGED_PATH = "checkpoints/gpt-oss-20b-coding-tui-merged"

MODEL_PATH = None
is_adapter = False
for path in CHECKPOINT_PRIORITY:
    if os.path.exists(path) and os.path.exists(os.path.join(path, "adapter_config.json")):
        MODEL_PATH = path
        is_adapter = True
        break

if MODEL_PATH is None and os.path.exists(MERGED_PATH):
    MODEL_PATH = MERGED_PATH
    is_adapter = False

if MODEL_PATH is None:
    print("✗ No checkpoint found. Train the model first.")
else:
    print(f"Loading model from: {MODEL_PATH}")
    print(f"  Type: {'LoRA adapter' if is_adapter else 'merged model'}")

    model = None
    base_name = "openai/gpt-oss-20b"

    # Try 1: Pre-quantized BNB 4-bit (avoids GptOssExperts BNB traversal issue)
    try:
        print("  Loading pre-quantized BNB 4-bit model...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            "unsloth/gpt-oss-20b-unsloth-bnb-4bit",
            max_seq_length=8192,
            dtype=None,
            load_in_4bit=False,
        )
        print("  Mode: BNB 4-bit (pre-quantized)")
    except Exception as e:
        print(f"  Pre-quantized BNB failed: {e}")

    # Try 2: bfloat16 without quantization
    if model is None:
        print("  Loading in bfloat16 (no quantization)...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            base_name,
            max_seq_length=8192,
            dtype=torch.bfloat16,
            load_in_4bit=False,
        )
        print("  Mode: bfloat16 (no quantization)")

    if is_adapter:
        print(f"  Applying LoRA adapter from {MODEL_PATH}...")
        model = PeftModel.from_pretrained(model, MODEL_PATH)

    FastLanguageModel.for_inference(model)
    print("✓ Model loaded!")

### 7.2 Test Against Failure Modes

In [ ]:
import sys, torch
sys.path.insert(0, "scripts")
from dataset_formatters.harmony import encode_harmony_messages

CODING_AGENT_DEV = (
    "You are a coding agent. Use tools to read files, write code, run tests, and "
    "complete programming tasks. Do not just analyze — always take action and produce "
    "working code. After making changes, verify they work by running the relevant tests. "
    "If a tool call fails, diagnose and retry with corrected parameters."
)

# Test prompts designed to expose each failure mode
TEST_PROMPTS = [
    # Test 1: Tool calling accuracy — should call read_file with valid path, NOT a made-up tool
    (
        "Failure Mode: Tool Calling",
        "Read the file at src/main.rs and fix any compilation errors you find.",
        ["read_file", "write_file", "run_command"],
    ),
    # Test 2: Follow-through — should NOT just say "I would need to look at..."
    (
        "Failure Mode: No Follow-Through",
        "Write a Python function called `binary_search(arr, target)` that returns the index of target in sorted arr, or -1 if not found. Add it to utils.py and write a pytest test for it.",
        None,
    ),
    # Test 3: Circular reasoning — model should take action, not loop
    (
        "Failure Mode: Circular Reasoning",
        "Analyze the codebase and suggest improvements. Then implement the most impactful one.",
        None,
    ),
    # Test 4: Context tracking — should remember the task mid-session
    (
        "Failure Mode: Context Loss",
        "I need you to refactor the authentication module. Start by reading auth.py, then identify the issues, then fix them one by one.",
        None,
    ),
]

def generate_response(prompt, tools=None, max_tokens=512):
    """Generate a response using Harmony format."""
    messages = [{"role": "user", "content": prompt}]
    if tools:
        tool_ctx = "\n".join(f"  - {t}(path: str)" for t in tools)
        messages[0]["content"] = (
            f"Available tools:\n{tool_ctx}\n\n" + messages[0]["content"]
        )
    formatted = encode_harmony_messages(
        messages,
        developer_instructions=CODING_AGENT_DEV,
        add_generation_prompt=True,
    )
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.3,
            do_sample=True,
            top_p=0.9,
        )
    return tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)


for label, prompt, tools in TEST_PROMPTS:
    print(f"\n{'=' * 64}")
    print(f"TEST: {label}")
    print(f"{'=' * 64}")
    print(f"Prompt: {prompt[:120]}...")
    print("-" * 64)
    response = generate_response(prompt, tools, max_tokens=384)
    print(response)
    print()

In [ ]:
# ── Custom Prompt ─────────────────────────────────────────────────────────────
CUSTOM_PROMPT = (
    "Read requirements.txt and install any missing packages, "
    "then run the test suite and fix any failing tests."
)

print(f"Custom prompt: {CUSTOM_PROMPT}")
print("=" * 64)
print(generate_response(CUSTOM_PROMPT, max_tokens=512))

## Step 8: Export

Merge the final adapter and export to HuggingFace safetensors + GGUF formats.

The GGUF file can be loaded directly into MacLean AI via llama-server
for Codex CLI integration testing.

### 8.1 Export to GGUF

In [ ]:
import os

tracker.start("export")

ADAPTER_PRIORITY = [
    "checkpoints/agent_sft_grpo/final",
    "checkpoints/agent_sft_ipo/final",
    "checkpoints/agent_sft/final",
    "checkpoints/tool_calling_sft/final",
]

adapter_path = None
for path in ADAPTER_PRIORITY:
    if os.path.exists(path):
        adapter_path = path
        break

if adapter_path is None:
    print("✗ No adapter checkpoint found.")
    tracker.fail("export")
else:
    export_dir = "checkpoints/gpt-oss-20b-coding-tui-export"
    print(f"Exporting adapter: {adapter_path}")
    print(f"Output: {export_dir}")
    print("=" * 60)

    !python scripts/19_merge_adapter.py \
        --adapter_path {adapter_path} \
        --output_dir {export_dir} \
        --export_formats hf gguf_q4

    drive_helper.backup(export_dir, "checkpoints/gpt-oss-20b-coding-tui-export")
    if DRIVE_MODE != "local":
        print("\nExport backed up to Drive.")

    tracker.complete("export")

### 8.2 QAT Export (Optional)

Quantisation-Aware Training for MXFP4 deployment.
Recovers 97-100% quality vs 59-89% with post-training quantisation.

In [ ]:
if not CONFIG.get("enable_qat_export"):
    print("QAT export disabled. Enable via widget toggle in Step 0.3.")
    print("\nQAT recovers 97-100% quality when deploying to MXFP4,")
    print("vs 59-89% with standard post-training quantisation (PTQ).")
else:
    import os
    export_dir = "checkpoints/gpt-oss-20b-coding-tui-export"
    qat_dir = "checkpoints/gpt-oss-20b-coding-tui-qat"

    if not os.path.exists(export_dir):
        print("✗ Run standard export (8.1) first.")
    else:
        print("Running QAT pass on merged model...")
        print("  This fine-tunes with MXFP4-aware quantisation at reduced LR (1e-5).")
        print("=" * 60)

        try:
            import modelopt.torch.quantization as mtq
            print("✓ nvidia-modelopt available")
            print("\nQAT pipeline (manual steps):")
            print(f"  1. Load merged BF16 model from {export_dir}")
            print(f"  2. mtq.quantize(model, config=mtq.MXFP4_DEFAULT_CFG)")
            print(f"  3. Fine-tune for ~100 steps at LR 1e-5")
            print(f"  4. Export to {qat_dir}")
        except ImportError:
            print("✗ nvidia-modelopt not installed.")
            print("  Install: pip install nvidia-modelopt")

### 8.3 Download GGUF

In [ ]:
IN_COLAB = "COLAB_GPU" in os.environ or os.path.exists("/content")

if IN_COLAB:
    from google.colab import files
    import glob, os

    export_dir = "checkpoints/gpt-oss-20b-coding-tui-export"
    gguf_files = glob.glob(os.path.join(export_dir, "**/*.gguf"), recursive=True)

    if gguf_files:
        gguf_path = gguf_files[0]
        size_gb = os.path.getsize(gguf_path) / (1024**3)
        print(f"Downloading: {os.path.basename(gguf_path)} ({size_gb:.1f} GB)")
        files.download(gguf_path)
    else:
        print("✗ No GGUF file found. Run export (8.1) first.")
else:
    print("Download not available outside Colab.")
    print("GGUF file is at: checkpoints/gpt-oss-20b-coding-tui-export/")

### 8.4 Upload to HuggingFace Hub

In [ ]:
# --- Configuration ---
HF_REPO_ID = ""  # e.g. "your-username/gpt-oss-20b-coding-tui-agent"
HF_PRIVATE = True

assert HF_REPO_ID, "Set HF_REPO_ID above before running this cell."

import os, glob
from huggingface_hub import HfApi

# Authenticate: try Colab Secrets first, then interactive login
try:
    from google.colab import userdata
    hf_token = userdata.get("HF_TOKEN")
    print("Using HF_TOKEN from Colab Secrets.")
except Exception:
    from huggingface_hub import login
    login()
    hf_token = None

api = HfApi(token=hf_token)
api.create_repo(repo_id=HF_REPO_ID, private=HF_PRIVATE, exist_ok=True)
print(f"Repo ready: https://huggingface.co/{HF_REPO_ID}")

# --- Model card ---
export_dir = "checkpoints/gpt-oss-20b-coding-tui-export"
hf_dir = os.path.join(export_dir, "hf")

model_card = """\
---
base_model: openai/gpt-oss-20b
tags:
  - coding-agent
  - tool-calling
  - codex-cli
  - gpt-oss
  - qlora
  - unsloth
  - grpo
  - tui
license: apache-2.0
pipeline_tag: text-generation
---

# GPT-OSS 20B Coding TUI Agent

Fine-tuned from [openai/gpt-oss-20b](https://huggingface.co/openai/gpt-oss-20b) for
use as a coding TUI agent (Codex CLI integration via MacLean AI + llama-server).

## Problem Statement

GPT-OSS 20B exhibits four failure modes when used as a coding agent:
1. Tool calling errors (invalid params, non-existent MCP servers)
2. No follow-through (analysis loops, never writes code)
3. Circular reasoning (repeating the same analysis)
4. Context loss (forgetting task state mid-session)

## Training Pipeline

1. **Tool Calling SFT** (rank 64) \u2014 Glaive + xLAM + Hermes in Harmony format
2. **Merge** \u2014 tool-calling adapter merged into base
3. **Agent SFT** (rank 128) \u2014 code-act, commitpack, editpackft + proxy log trajectories
4. **IPO** \u2014 decisive action preferred over circular analysis
5. **GRPO RL** \u2014 execution-grounded: code compiles, tests pass, no loops

Trained with [Unsloth](https://github.com/unslothai/unsloth) QLoRA.

## Deployment

Designed for deployment via [llama-server](https://github.com/ggerganov/llama.cpp)
with the [claude-proxy-v2](https://github.com/rmarnold/claude-proxy-v2) translation layer
for Codex CLI (OpenAI Responses API).

## GGUF

A quantised GGUF file is included for use with llama.cpp.
""".format()

readme_path = os.path.join(hf_dir, "README.md")
os.makedirs(hf_dir, exist_ok=True)
with open(readme_path, "w") as f:
    f.write(model_card)
print(f"Wrote model card to {readme_path}")

# --- Upload HF safetensors model ---
assert os.path.isdir(hf_dir), f"HF export dir not found: {hf_dir}. Run export (8.1) first."
print(f"Uploading HF model from {hf_dir} ...")
api.upload_folder(
    folder_path=hf_dir,
    repo_id=HF_REPO_ID,
    commit_message="Upload GPT-OSS 20B Coding TUI Agent (tool-calling + agent-SFT + IPO + GRPO)",
    token=hf_token,
)
print("HF model uploaded.")

# --- Upload GGUF file ---
gguf_files = glob.glob(os.path.join(export_dir, "**/*.gguf"), recursive=True)
if gguf_files:
    gguf_path = gguf_files[0]
    gguf_name = os.path.basename(gguf_path)
    size_gb = os.path.getsize(gguf_path) / (1024**3)
    print(f"Uploading GGUF: {gguf_name} ({size_gb:.1f} GB) ...")
    api.upload_file(
        path_or_fileobj=gguf_path,
        path_in_repo=gguf_name,
        repo_id=HF_REPO_ID,
        commit_message=f"Upload GGUF quantisation ({gguf_name})",
        token=hf_token,
    )
    print("GGUF uploaded.")
else:
    print("No GGUF file found \u2014 skipping. Run export (8.1) to generate one.")

print(f"\nDone! View your model at: https://huggingface.co/{HF_REPO_ID}")

---
## Training Complete!

Your GPT-OSS 20B Coding TUI Agent is trained and ready for Codex CLI integration.

**Pipeline summary:**
1. Tool Calling SFT: Glaive (113K) + xLAM (60K) + Hermes — correct tool schemas and parameter formatting
2. Merge: tool-calling adapter fused into base weights
3. Agent SFT: code-act + commitpack + editpackft + real proxy log trajectories (most valuable)
4. IPO: decisive action preferred over circular analysis (hh-rlhf + code feedback)
5. GRPO RL: execution-grounded — rewards for compiling code and passing tests, penalises loops

**Outputs:**
- Checkpoints: `checkpoints/agent_sft_{ipo,grpo}/final`
- Evaluation: `evals/coding_tui_agent/metrics.json`
- Exported model: `checkpoints/gpt-oss-20b-coding-tui-export/`
- All backed up to Google Drive: `gpt-oss-20b-coding-tui/`

**MacLean AI integration:**
- Copy the exported GGUF to your MacLean AI model directory
- Select it in the Model Browser
- Enable Codex CLI support in Settings
- The proxy translation layer (`--translate-anthropic`) handles the Anthropic↔OpenAI format conversion
- Test with: `codex "Read src/main.rs and fix any compilation errors"`

**Next steps:**
- Review evaluation metrics in Step 6
- Test against each failure mode in Step 7
- If circular reasoning persists: increase GRPO steps or add more no-action penalisation
- If tool calling is still poor: increase tool_calling_sft steps or add more xLAM data

In [ ]:
# Disconnect and release GPU runtime to stop billing
try:
    from google.colab import runtime
    runtime.unassign()
except ImportError:
    print("Not in Colab — no runtime to release.")